# Unión de datos scrapeados, Dataframe final 

In [1]:
!ls -lrt

total 33728
-rw-r--r--  1 prealfer  staff    420474 31 ago 19:15 TFM_scraping.ipynb
-rw-r--r--  1 prealfer  staff         0  5 sep 23:51 README.md
-rw-r--r--  1 prealfer  staff         0  5 sep 23:51 README.txt
-rw-r--r--  1 prealfer  staff    184491  6 sep 23:02 car_models.csv
-rw-r--r--  1 prealfer  staff   2352266 25 sep 23:32 listado_modelos_3.txt
-rw-r--r--  1 prealfer  staff     87691 28 sep 00:34 precios_versiones
-rw-r--r--  1 prealfer  staff   1584860 30 sep 18:04 versiones_con_precio
-rw-r--r--  1 prealfer  staff     15285 30 sep 20:02 maleteros
-rw-r--r--  1 prealfer  staff     16680 30 sep 20:06 Scrapeo-Maleteros.ipynb
-rw-r--r--  1 prealfer  staff    500156 30 sep 20:07 Script scrapping - Dataframe.ipynb
-rw-r--r--  1 prealfer  staff  12086763  1 oct 17:29 Construcción Dataframe definitivo.ipynb


In [113]:
import pandas as pd
import numpy as np

In [114]:
pd.set_option('display.max_columns', None)

In [188]:
datos = pd.read_csv('Dataframe_limpio.csv',header=0)
datos = datos.drop('Unnamed: 0',axis=1)
datos.head(40)

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,airbags,garantia,cilindrada,capacidad deposito,marchas,max_vel,0_a_100,rpm,par,control traccion,control estabilidad,estrellas NCAP,alarma,cierre centralizado,cierre automatico,tipo llave,start/stop,CD/mp3/dvd,carroceria,peso
0,Abarth,124 Spider,Abarth 1.4 170,35.093€,2 puertas,Gasolina,Tracción trasera,170 cv,Manual,6 marchas,148 gr/km,703.1 km,6.4 l/100km,0,24 meses,1.368 cc,45 lts.,6,232 km/h,6.80 s.,5000 rpm,250Nm,Sí,No,-,No,No,No,NaN,No,No,4054/1740/1233,NaN
1,Abarth,124 Spider,Abarth 1.4 Scorpione 170,36.511€,2 puertas,Gasolina,Tracción trasera,170 cv,Manual,6 marchas,148 gr/km,703.1 km,6.4 l/100km,0,24 meses,1.368 cc,45 lts.,6,232 km/h,6.80 s.,5000 rpm,250Nm,Sí,No,-,No,No,No,NaN,No,No,4054/1740/1233,NaN
2,Abarth,124 Spider,Abarth 1.4 Aut. 170,37.961€,2 puertas,Gasolina,Tracción trasera,170 cv,Automática secuencial,6 marchas,153 gr/km,681.8 km,6.6 l/100km,0,24 meses,1.368 cc,45 lts.,6,229 km/h,6.90 s.,5000 rpm,250Nm,Sí,No,-,No,No,No,NaN,No,No,4054/1740/1233,NaN
3,Abarth,124 Spider,Abarth 1.4 Scorpione Aut. 170,38.511€,2 puertas,Gasolina,Tracción trasera,170 cv,Automática secuencial,6 marchas,153 gr/km,681.8 km,6.6 l/100km,0,24 meses,1.368 cc,45 lts.,6,229 km/h,6.90 s.,5000 rpm,250Nm,Sí,No,-,No,No,No,NaN,No,No,4054/1740/1233,NaN
4,Abarth,595,Abarth 1.4T-Jet 145,15.670€,3 puertas,Gasolina,Tracción delantera,145 cv,Manual,5 marchas,139 gr/km,583.3 km,6.0 l/100km,0,0 meses,1.368 cc,35 lts.,5,205 km/h,7.90 s.,5500 rpm,206Nm,Sí,No,-,No,No,No,NaN,No,No,3660/1627/1480,NaN
5,Abarth,595,Abarth C 1.4T-Jet 145,17.316€,2 puertas,Gasolina,Tracción delantera,145 cv,Manual,5 marchas,139 gr/km,583.3 km,6.0 l/100km,0,0 meses,1.368 cc,35 lts.,5,205 km/h,7.90 s.,5500 rpm,206Nm,Sí,No,-,No,No,No,NaN,No,No,3660/1627/1480,NaN
6,Abarth,595,Abarth 1.4T-Jet Turismo 165,20.497€,3 puertas,Gasolina,Tracción delantera,165 cv,Manual,5 marchas,139 gr/km,583.3 km,6.0 l/100km,0,0 meses,1.368 cc,35 lts.,5,210 km/h,7.40 s.,5500 rpm,230Nm,Sí,No,-,No,No,No,NaN,No,No,3660/1627/1485,NaN
7,Abarth,595,Abarth 1.4T-Jet 145 Aut.,21.086€,3 puertas,Gasolina,Tracción delantera,145 cv,Automática secuencial,5 marchas,139 gr/km,583.3 km,6.0 l/100km,0,0 meses,1.368 cc,35 lts.,5,205 km/h,7.90 s.,5500 rpm,206Nm,Sí,No,-,No,No,No,NaN,No,No,3660/1627/1480,NaN
8,Abarth,595,Abarth 1.4T-Jet Pista 160,21.861€,3 puertas,Gasolina,Tracción delantera,160 cv,Manual,5 marchas,139 gr/km,583.3 km,6.0 l/100km,0,0 meses,1.368 cc,35 lts.,5,210 km/h,7.40 s.,5500 rpm,230Nm,Sí,No,-,No,No,No,NaN,No,No,3660/1627/1485,NaN
9,Abarth,595,Abarth C 1.4T-Jet Turismo 165,22.208€,2 puertas,Gasolina,Tracción delantera,165 cv,Manual,5 marchas,139 gr/km,583.3 km,6.0 l/100km,0,0 meses,1.368 cc,35 lts.,5,210 km/h,7.40 s.,5500 rpm,230Nm,Sí,No,-,No,No,No,NaN,No,No,3660/1627/1485,NaN


In [524]:
maleteros = pd.read_csv('maleteros')
maleteros = maleteros.drop('Unnamed: 0',axis=1)
maleteros.head()

,Marca,Modelo,CapacidadMaletero
0,ALFA ROMEO,Alfa Romeo MiTo,270
1,ALFA ROMEO,Alfa Romeo 4C,110
2,ALFA ROMEO,Alfa Romeo Giulietta,350
3,ALFA ROMEO,Alfa Romeo Giulia,480
4,ALFA ROMEO,Nuevo Alfa Romeo Stelvio 2017,525


## Unificamos en mayúsculas 

In [525]:
datos['marca'] = datos['marca'].apply(lambda x: x.upper().strip())
datos['modelo'] = datos['modelo'].apply(lambda x: x.upper().strip())
maleteros['Modelo'] = maleteros['Modelo'].apply(lambda x: x.upper())

### Creamos columna que contiene marca y modelo 

In [191]:
datos['mm']=datos['marca']+' '+datos['modelo']

In [526]:
maleteros.columns = ['Marca','Modelo','Maletero']

### Renombramos columnas para quitar '-' 

In [193]:
#datos.rename(columns={'marca':'marca', 'modelo':'modelo', 'version':'version', 'precio':'precio', 'puertas':'puertas', 'combustible':'combustible',
       'traccion':'traccion', 'cv':'cv', 'cambio':'cambio', 'n_marchas':'n_marchas', 'co2':'co2', 'autonomia':'autonomia',
       'consumo':'consumo', 'airbags':'airbags', 'garantia':'garantia', 'cilindrada':'cilindrada',
       'capacidad deposito':'capacidad deposito', 'marchas':'marchas', 'max_vel':'max_vel', '0_a_100':'0_a_100', 'rpm':'rpm', 'par':'par',
       'control traccion':'control traccion', 'control estabilidad':'control estabilidad', 'estrellas NCAP':'estrellas NCAP',
       'alarma':'alarma', 'cierre centralizado':'cierre centralizado', 'cierre automatico':'cierre automatico', 'tipo llave':'tipo llave',
       'start/stop':'start/stop', 'CD/mp3/dvd':'CD/mp3/dvd', 'carroceria':'carroceria', 'peso':'peso', 'marca-modelo':'mm',
       'maletero':'maletero'}, inplace=True)

In [515]:
maleteros.head()

,Marca,Modelo,Maletero
0,ALFA ROMEO,ALFA ROMEO MITO,270
1,ALFA ROMEO,ALFA ROMEO 4C,110
2,ALFA ROMEO,ALFA ROMEO GIULIETTA,350
3,ALFA ROMEO,ALFA ROMEO GIULIA,480
4,ALFA ROMEO,NUEVO ALFA ROMEO STELVIO 2017,525


In [527]:
modelos_maleteros = []
modelos_maleteros = maleteros['Modelo']
len(modelos_maleteros)

427

In [196]:
modelos_data = []
modelos_data = datos['mm']
len(modelos_data)

5932

## Limpiamos los modelos de caracteres extraños y creamos nueva columna limpia

In [197]:
modelos_data = [mod.replace('NUEVO ','').replace('\xc3\xb3','O').replace('\xc3\xa9','E') for mod in modelos_data]
modelos_data

['ABARTH 124 SPIDER',
 'ABARTH 124 SPIDER',
 'ABARTH 124 SPIDER',
 'ABARTH 124 SPIDER',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'ABARTH 595',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A1',
 'AUDI A3',


In [198]:
datos['MODELOS'] = modelos_data

In [528]:
modelos_maleteros = [mod.replace('NUEVO ','').replace(' 2017','').replace(' 2018','').replace('-BENZ','').replace('!','').replace('\xc5\xa0','S').replace('\xc3\xb3','O').replace('!','').replace('\xc3\xa9','E').replace('\xc3\xab','E') for mod in modelos_maleteros]

modelos_maleteros

['ALFA ROMEO MITO',
 'ALFA ROMEO 4C',
 'ALFA ROMEO GIULIETTA',
 'ALFA ROMEO GIULIA',
 'ALFA ROMEO STELVIO',
 'ASTON MARTIN V12 VANTAGE S',
 'ASTON MARTIN DB11',
 'ASTON MARTIN RAPIDE S',
 'ASTON MARTIN VANQUISH COUPE',
 'ASTON MARTIN VANQUISH VOLANTE',
 'AUDI A1',
 'AUDI A1 SPORTBACK',
 'AUDI S1',
 'AUDI A3',
 'AUDI A3 SPORTBACK',
 'AUDI A3 CABRIO',
 'AUDI A3 SEDAN',
 'AUDI S3',
 'AUDI A4',
 'AUDI A4 AVANT',
 'AUDI A4 ALLROAD QUATTRO',
 'AUDI S4',
 'AUDI A5 COUPE',
 'AUDI A5 SPORTBACK',
 'AUDI S5',
 'AUDI A6',
 'AUDI A6 AVANT',
 'AUDI A7 SPORTBACK',
 'AUDI A8',
 'AUDI A8 L',
 'AUDI TT COUPE',
 'AUDI R8',
 'AUDI Q2',
 'AUDI Q3',
 'AUDI Q5',
 'AUDI Q7',
 'BENTLEY CONTINENTAL GT',
 'BENTLEY CONTINENTAL GT CONVERTIBLE',
 'BENTLEY BENTAYGA',
 'BENTLEY FLYING SPUR',
 'BENTLEY MULSANNE',
 'BMW SERIE 1',
 'BMW I3',
 'BMW SERIE 2 COUPE',
 'BMW SERIE 2 CABRIO',
 'BMW SERIE 2 ACTIVE TOURER',
 'BMW SERIE 2 GRAN TOURER',
 'BMW M2 COUPE',
 'BMW SERIE 3 BERLINA',
 'BMW SERIE 3 TOURING',
 'BMW M3 BERL

In [529]:
maleteros['MODELOS'] =  modelos_maleteros

### Eliminamos columna obsoleta 

In [530]:
maleteros = maleteros.drop('Modelo',axis=1)

In [531]:
datos = datos.drop('mm', axis=1)

ValueError: labels ['mm'] not contained in axis

In [532]:
maleteros.rename(columns={'Marca': 'marca', 'marca-modelo': 'modelo', 'Maletero': 'maletero'}, inplace=True)

# Creamos función que introduce la columna maleteros con sus datos

In [447]:
def f(row):
  if row["MODELOS"] not in modelos_maleteros:
    return 'hola'
  else:
    return maleteros[maleteros['MODELOS']==row["MODELOS"]].maletero.values[0]

In [448]:
datos["maleteroF"] = datos.apply(f, axis=1)

TypeError: ('list indices must be integers, not str', u'occurred at index 32')

In [456]:
datos[32,:]

TypeError: unhashable type

In [207]:
len(datos.puertas.unique())

34

## Guardamos el archivo final. 
### Faltaría:
#### Datos de los maleteros que restan.
#### Limpiar unidades o strings no útiles para atributos y pasarlos al 'header'.
#### Normalizar datos. 

In [208]:
!head -100 datos_con_maletero.csv

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,airbags,garantia,cilindrada,capacidad deposito,marchas,max_vel,0_a_100,rpm,par,control traccion,control estabilidad,estrellas NCAP,alarma,cierre centralizado,cierre automatico,tipo llave,start/stop,CD/mp3/dvd,carroceria,peso,MODELOS,maletero
0,ABARTH,124 SPIDER,Abarth  1.4 170,35.093€,2 puertas ,Gasolina ,Tracción trasera ,170 cv ,Manual ,6 marchas ,148 gr/km ,703.1 km ,6.4 l/100km ,0 ,24 meses ,1.368 cc ,45 lts. ,6 ,232 km/h ,6.80 s.,5000 rpm,250Nm ,Sí,No,-,No,No,No,,No ,No,4054/1740/1233,,ABARTH 124 SPIDER,No hay
1,ABARTH,124 SPIDER,Abarth  1.4 Scorpione 170,36.511€,2 puertas ,Gasolina ,Tracción trasera ,170 cv ,Manual ,6 marchas ,148 gr/km ,703.1 km ,6.4 l/100km ,0 ,24 meses ,1.368 cc ,45 lts. ,6 ,232 km/h ,6.80 s.,5000 rpm,250Nm ,Sí,No,-,No,No,No,,No ,No,4054/1740/1233,,ABARTH 124 SPIDER,No hay
2,ABARTH,124 SPIDER,Abarth  1.4 Aut. 170,37.961€,2 puertas ,Gasolina ,Tracción trasera ,17

In [209]:
datos.to_csv('datos_con_maletero.csv', encoding='utf-8')

# Adaptación y normalización.

In [210]:
df = pd.read_csv('datos_con_maletero.csv', sep=',')
df = df.drop('Unnamed: 0',axis=1)

In [211]:
df.tail(100)

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,airbags,garantia,cilindrada,capacidad deposito,marchas,max_vel,0_a_100,rpm,par,control traccion,control estabilidad,estrellas NCAP,alarma,cierre centralizado,cierre automatico,tipo llave,start/stop,CD/mp3/dvd,carroceria,peso,MODELOS,maletero
5832,VOLVO,V60,D3 Summum Aut. 150,28.059€,5 puertas,Diesel,Tracción delantera,150 cv,Automática secuencial,6 marchas,114 gr/km,1.569.8 km,4.3 l/100km,0,24 meses,1.969 cc,67.5 lts.,6,210 km/h,9.10 s.,3750 rpm,320Nm,Sí,No,-,No,No,No,NaN,No,No,4635/1865/1484,NaN,VOLVO V60,430
5833,VOLVO,V60,D3 R-Design Momentum Aut. 150,28.390€,5 puertas,Diesel,Tracción delantera,150 cv,Automática secuencial,6 marchas,114 gr/km,1.569.8 km,4.3 l/100km,0,24 meses,1.969 cc,67.5 lts.,6,210 km/h,9.10 s.,3750 rpm,320Nm,Sí,No,-,No,No,No,NaN,No,No,4635/1865/1484,NaN,VOLVO V60,430
5834,VOLVO,V60,D4 Summum 190,28.645€,5 puertas,Diesel,Tracción delantera,190 cv,Manual,6 marchas,104 gr/km,1.687.5 km,4.0 l/100km,0,24 meses,1.969 cc,67.5 lts.,6,225 km/h,7.70 s.,4250 rpm,400Nm,Sí,No,-,No,No,No,NaN,No,No,4635/1865/1484,NaN,VOLVO V60,430
5835,VOLVO,V60,T4 Summum 190,28.950€,5 puertas,Gasolina,Tracción delantera,190 cv,Manual,6 marchas,135 gr/km,1.163.8 km,5.8 l/100km,0,24 meses,1.969 cc,67.5 lts.,6,225 km/h,7.30 s.,4700 rpm,300Nm,Sí,No,-,No,No,No,NaN,No,No,4635/1865/1484,NaN,VOLVO V60,430
5836,VOLVO,V60,D4 R-Design Momentum 190,28.975€,5 puertas,Diesel,Tracción delantera,190 cv,Manual,6 marchas,104 gr/km,1.687.5 km,4.0 l/100km,0,24 meses,1.969 cc,67.5 lts.,6,225 km/h,7.70 s.,4250 rpm,400Nm,Sí,No,-,No,No,No,NaN,No,No,4635/1865/1484,NaN,VOLVO V60,430
5837,VOLVO,V60,T4 Kinetic 190,29.135€,5 puertas,Gasolina,Tracción delantera,190 cv,Manual,6 marchas,135 gr/km,1.163.8 km,5.8 l/100km,0,24 meses,1.969 cc,67.5 lts.,6,225 km/h,7.30 s.,4700 rpm,300Nm,Sí,No,-,No,No,No,NaN,No,No,4635/1865/1484,NaN,VOLVO V60,430
5838,VOLVO,V60,T4 Momentum Aut. 190,29.208€,5 puertas,Gasolina,Tracción delantera,190 cv,Automática secuencial,6 marchas,136 gr/km,1.163.8 km,5.8 l/100km,0,24 meses,1.969 cc,67.5 lts.,6,225 km/h,7.30 s.,4700 rpm,300Nm,Sí,No,-,No,No,No,NaN,No,No,4635/1865/1484,NaN,VOLVO V60,430
5839,VOLVO,V60,D4 Momentum Aut. 190,29.223€,5 puertas,Diesel,Tracción delantera,190 cv,Automática secuencial,8 marchas,116 gr/km,1.534.1 km,4.4 l/100km,0,24 meses,1.969 cc,67.5 lts.,8,225 km/h,7.70 s.,4250 rpm,400Nm,Sí,No,-,No,No,No,NaN,No,No,4635/1865/1484,NaN,VOLVO V60,430
5840,VOLVO,V60,T4 R-Design Momentum 190,29.294€,5 puertas,Gasolina,Tracción delantera,190 cv,Manual,6 marchas,135 gr/km,1.163.8 km,5.8 l/100km,0,24 meses,1.969 cc,67.5 lts.,6,225 km/h,7.30 s.,4700 rpm,300Nm,Sí,No,-,No,No,No,NaN,No,No,4635/1865/1484,NaN,VOLVO V60,430
5841,VOLVO,V60,D3 Kinetic Aut. 150,29.297€,5 puertas,Diesel,Tracción delantera,150 cv,Automática secuencial,6 marchas,114 gr/km,1.569.8 km,4.3 l/100km,0,24 meses,1.969 cc,67.5 lts.,6,210 km/h,9.10 s.,3750 rpm,320Nm,Sí,No,-,No,No,No,NaN,No,No,4635/1865/1484,NaN,VOLVO V60,430


## Eliminar vehículos comerciales 

In [212]:
df.marca.unique()

array(['ABARTH', 'AUDI', 'BMW', 'CITROEN', 'DACIA', 'DS', 'FIAT', 'FORD',
       'HONDA', 'HYUNDAI', 'INFINITI', 'JAGUAR', 'JEEP', 'KIA', 'LEXUS',
       'MASERATI', 'MAZDA', 'MERCEDES', 'MINI', 'MITSUBISHI', 'MORGAN',
       'NISSAN', 'OPEL', 'PEUGEOT', 'PORSCHE', 'RENAULT', 'SEAT', 'SKODA',
       'SMART', 'SSANGYONG', 'SUBARU', 'SUZUKI', 'TESLA', 'TOYOTA',
       'VOLKSWAGEN', 'VOLVO'], dtype=object)

In [213]:
df[df['marca']=='VOLKSWAGEN'].modelo.unique()

array(['AMAROK', 'ARTEON', 'BEETLE', 'CADDY', 'CALIFORNIA', 'CARAVELLE',
       'CRAFTER', 'GOLF', 'GOLF ALLTRACK', 'MULTIVAN', 'PASSAT',
       'PASSAT ALLTRACK', 'POLO', 'SHARAN', 'TIGUAN', 'TOUAREG', 'TOURAN',
       'TRANSPORTER', 'UP', 'UP COMERCIAL'], dtype=object)

In [214]:
borrar_volkswagen = ['CADDY','CARAVELLE','CRAFTER','TRANSPORTER', 'UP COMERCIAL']

In [215]:
borrar_toyota = ['LAND CRUISER COMERCIAL', 'PROACE', 'PROACE VERSO']

In [216]:
borrar_seat = ['IBIZA COMERCIAL']

In [217]:
borrar_renault = ['KANGOO', 'MASTER', 'TRAFIC']

In [218]:
borrar_peugeot = ['BIPPER COMERCIAL', 'BOXER', 'EXPERT', 'PARTNER','TRAVELLER', 'TRAVELLER M1']

In [219]:
borrar_opel = ['COMBO', 'MOVANO', 'VIVARO']

In [220]:
borrar_nissan = ['EVALIA','NT400', 'NV200', 'NV300', 'NV400']

In [221]:
borrar_mercedes = ['CITAN', 'SPRINTER', 'VITO']

In [222]:
borrar_ford = ['CONNECT', 'TOURNEO CONNECT', 'TOURNEO COURIER',
       'TOURNEO CUSTOM', 'TRANSIT', 'TRANSIT COURIER', 'TRANSIT CUSTOM']

In [223]:
borrar_dacia = ['DOKKER', 'DOKKER COMERCIAL', 'LODGY']

In [224]:
borrar_citroen = ['BERLINGO','C3 COMERCIAL', 'JUMPER', 'JUMPY', 'NEMO', 'NEMO COMERCIAL', 'SPACETOURER']

In [225]:
borrar_fiat = ['DOBL\xc3\xb3', 'DUCATO',
       'FIORINO', 'FIORINO COMERCIAL', 'FULLBACK', 'TALENTO']

In [226]:
modelos_borrar = borrar_volkswagen + borrar_toyota + borrar_seat + borrar_renault + borrar_peugeot + borrar_opel + borrar_nissan + borrar_mercedes + borrar_ford + borrar_dacia + borrar_citroen + borrar_fiat

In [227]:
for modelo in modelos_borrar:
    df = df[df['modelo']!=modelo]

In [292]:
df.tail(100)

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,airbags,garantia,cilindrada,capacidad deposito,marchas,max_vel,0_a_100,rpm,par,control traccion,control estabilidad,estrellas NCAP,alarma,cierre centralizado,cierre automatico,tipo llave,start/stop,CD/mp3/dvd,carroceria,peso,MODELOS,maletero
5832,VOLVO,V60,D3 Summum Aut. 150,28059,5,Diesel,delantera,150,Automática secuencial,6,114,1.569.8,4.3,0,24,1.969,67.5,6,210,9.1,3750,320.0,Sí,No,-,No,No,No,NaN,No,No,4635/1865/1484,NaN,VOLVO V60,430
5833,VOLVO,V60,D3 R-Design Momentum Aut. 150,28390,5,Diesel,delantera,150,Automática secuencial,6,114,1.569.8,4.3,0,24,1.969,67.5,6,210,9.1,3750,320.0,Sí,No,-,No,No,No,NaN,No,No,4635/1865/1484,NaN,VOLVO V60,430
5834,VOLVO,V60,D4 Summum 190,28645,5,Diesel,delantera,190,Manual,6,104,1.687.5,4.0,0,24,1.969,67.5,6,225,7.7,4250,400.0,Sí,No,-,No,No,No,NaN,No,No,4635/1865/1484,NaN,VOLVO V60,430
5835,VOLVO,V60,T4 Summum 190,28950,5,Gasolina,delantera,190,Manual,6,135,1.163.8,5.8,0,24,1.969,67.5,6,225,7.3,4700,300.0,Sí,No,-,No,No,No,NaN,No,No,4635/1865/1484,NaN,VOLVO V60,430
5836,VOLVO,V60,D4 R-Design Momentum 190,28975,5,Diesel,delantera,190,Manual,6,104,1.687.5,4.0,0,24,1.969,67.5,6,225,7.7,4250,400.0,Sí,No,-,No,No,No,NaN,No,No,4635/1865/1484,NaN,VOLVO V60,430
5837,VOLVO,V60,T4 Kinetic 190,29135,5,Gasolina,delantera,190,Manual,6,135,1.163.8,5.8,0,24,1.969,67.5,6,225,7.3,4700,300.0,Sí,No,-,No,No,No,NaN,No,No,4635/1865/1484,NaN,VOLVO V60,430
5838,VOLVO,V60,T4 Momentum Aut. 190,29208,5,Gasolina,delantera,190,Automática secuencial,6,136,1.163.8,5.8,0,24,1.969,67.5,6,225,7.3,4700,300.0,Sí,No,-,No,No,No,NaN,No,No,4635/1865/1484,NaN,VOLVO V60,430
5839,VOLVO,V60,D4 Momentum Aut. 190,29223,5,Diesel,delantera,190,Automática secuencial,8,116,1.534.1,4.4,0,24,1.969,67.5,8,225,7.7,4250,400.0,Sí,No,-,No,No,No,NaN,No,No,4635/1865/1484,NaN,VOLVO V60,430
5840,VOLVO,V60,T4 R-Design Momentum 190,29294,5,Gasolina,delantera,190,Manual,6,135,1.163.8,5.8,0,24,1.969,67.5,6,225,7.3,4700,300.0,Sí,No,-,No,No,No,NaN,No,No,4635/1865/1484,NaN,VOLVO V60,430
5841,VOLVO,V60,D3 Kinetic Aut. 150,29297,5,Diesel,delantera,150,Automática secuencial,6,114,1.569.8,4.3,0,24,1.969,67.5,6,210,9.1,3750,320.0,Sí,No,-,No,No,No,NaN,No,No,4635/1865/1484,NaN,VOLVO V60,430


In [290]:
df.combustible.unique()

array(['Gasolina ', 'Diesel ', 'Diesel y corriente el\xc3\xa9ctrica ',
       'Corriente el\xc3\xa9ctrica ',
       'Gasolina y corriente el\xc3\xa9ctrica ', 'Gasolina/gas '], dtype=object)

## Limpieza y codificacion


In [250]:
df.garantia.unique()

array(['24 meses ', '0 meses ', '36 meses ', '60 meses ', '84 meses ',
       '12 meses ', '72 meses '], dtype=object)

In [231]:
df['precio'] = df['precio'].apply(lambda x: x.replace('€','').replace('.',''))


In [235]:
df['puertas'] = df['puertas'].apply(lambda x: x.replace(' puertas',''))

In [305]:
df['puertas'] = df['puertas'].astype('category')

In [236]:
df['cv'] = df['cv'].apply(lambda x: x.replace(' cv',''))

In [260]:
df['cv'] = df['cv'].astype(int)

In [238]:
df['traccion'] = df['traccion'].apply(lambda x: x.replace('Tracción ',''))

In [306]:
df['traccion'] = df['traccion'].astype('category')

In [239]:
df['n_marchas'] = df['n_marchas'].apply(lambda x: x.replace(' marchas',''))

In [307]:
df['n_marchas'] = df['n_marchas'].astype('category')

In [240]:
df['co2'] = df['co2'].apply(lambda x: x.replace(' gr/km',''))

In [261]:
df['co2'] = df['co2'].astype(int)

In [241]:
df['autonomia'] = df['autonomia'].apply(lambda x: x.replace(' km',''))

In [242]:
df['consumo'] = df['consumo'].apply(lambda x: x.replace(' l/100km',''))

In [266]:
df['consumo'] = df['consumo'].astype(float)

In [243]:
df['cilindrada'] = df['cilindrada'].apply(lambda x: x.replace(' cc',''))

In [277]:
df['cilindrada'] = df['cilindrada'].astype(float)

In [244]:
df['capacidad deposito'] = df['capacidad deposito'].apply(lambda x: x.replace(' lts.',''))

In [281]:
df['capacidad deposito'] = df['capacidad deposito'].astype(float)

In [245]:
df['max_vel'] = df['max_vel'].apply(lambda x: x.replace(' km/h',''))

In [271]:
df['max_vel'] = df['max_vel'].astype(int)

In [246]:
df['0_a_100'] = df['0_a_100'].apply(lambda x: x.replace(' s.',''))

In [272]:
df['0_a_100'] = df['0_a_100'].astype(float)

In [248]:
df['rpm'] = df['rpm'].apply(lambda x: x.replace(' rpm',''))

In [273]:
df['rpm'] = df['rpm'].astype(int)

In [247]:
df['par'] = df['par'].apply(lambda x: x.replace('Nm',''))

In [275]:
df['par'] = df['par'].astype(float)

In [251]:
df['garantia'] = df['garantia'].apply(lambda x: x.replace(' meses','').replace('.',''))

In [278]:
df['garantia'] = df['garantia'].astype(int)

In [309]:
df['cambio'] = df['cambio'].astype('category')

In [316]:
df['combustible'] = df['combustible'].astype('category')

In [317]:
df.dtypes

marca                   object
modelo                  object
version                 object
precio                   int64
puertas               category
combustible           category
traccion              category
cv                       int64
cambio                category
n_marchas             category
co2                      int64
autonomia               object
consumo                float64
garantia                 int64
cilindrada             float64
capacidad deposito     float64
max_vel                  int64
0_a_100                float64
rpm                      int64
par                    float64
estrellas NCAP          object
carroceria              object
MODELOS                 object
maletero                object
dtype: object

In [318]:
maleteros.head

<bound method DataFrame.head of             marca  maletero                        MODELOS
0      ALFA ROMEO       270                ALFA ROMEO MITO
1      ALFA ROMEO       110                  ALFA ROMEO 4C
2      ALFA ROMEO       350           ALFA ROMEO GIULIETTA
3      ALFA ROMEO       480              ALFA ROMEO GIULIA
4      ALFA ROMEO       525             ALFA ROMEO STELVIO
5    ASTON_MARTIN       300     ASTON MARTIN V12 VANTAGE S
6    ASTON_MARTIN       270              ASTON MARTIN DB11
7    ASTON_MARTIN       317          ASTON MARTIN RAPIDE S
8    ASTON_MARTIN       368    ASTON MARTIN VANQUISH COUPE
9    ASTON_MARTIN       279  ASTON MARTIN VANQUISH VOLANTE
10           AUDI       270                        AUDI A1
11           AUDI       270              AUDI A1 SPORTBACK
12           AUDI       210                        AUDI S1
13           AUDI       365                        AUDI A3
14           AUDI       365              AUDI A3 SPORTBACK
15           AUDI       

In [311]:
df.columns

Index([u'marca', u'modelo', u'version', u'precio', u'puertas', u'combustible',
       u'traccion', u'cv', u'cambio', u'n_marchas', u'co2', u'autonomia',
       u'consumo', u'airbags', u'garantia', u'cilindrada',
       u'capacidad deposito', u'marchas', u'max_vel', u'0_a_100', u'rpm',
       u'par', u'control traccion', u'control estabilidad', u'estrellas NCAP',
       u'alarma', u'cierre centralizado', u'cierre automatico', u'tipo llave',
       u'start/stop', u'CD/mp3/dvd', u'carroceria', u'peso', u'MODELOS',
       u'maletero'],
      dtype='object')

In [312]:
cols_to_drop = ['airbags','marchas','control traccion','control estabilidad','alarma','cierre centralizado','tipo llave','peso','cierre automatico','start/stop','CD/mp3/dvd']

In [313]:
df = df.drop(cols_to_drop, axis=1)

In [314]:
df.describe()

,precio,cv,co2,consumo,garantia,cilindrada,capacidad deposito,max_vel,0_a_100,rpm,par
count,4670.000000,4670.000000,4670.000000,4670.000000,4670.000000,4670.000000,4670.000000,4670.000000,4670.000000,4670.000000,4670.000000
mean,32946.096788,164.370878,129.664882,5.295375,30.447109,92.293485,56.060771,196.898715,9.239850,4317.740899,297.530600
std,28370.150896,89.177477,38.830810,1.716659,14.601690,283.500931,13.326035,44.996930,3.216411,1505.377025,139.779877
min,6448.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,17601.750000,115.000000,108.000000,4.300000,24.000000,1.499000,50.000000,182.000000,7.800000,3750.000000,205.000000
50%,24951.000000,145.000000,121.000000,5.000000,24.000000,1.968000,55.000000,198.000000,9.700000,4000.000000,280.000000
75%,36272.000000,182.000000,144.000000,6.000000,36.000000,2.143000,64.000000,218.000000,11.100000,5500.000000,380.000000
max,291180.000000,700.000000,397.000000,17.000000,84.000000,999.000000,100.000000,350.000000,20.000000,9900.000000,1100.000000


In [304]:
df[df['0_a_100']==0].modelo.unique()

array(['A4', 'SERIE 7', 'LOGAN', 'SANDERO', 'TIPO', 'B-MAX', 'C-MAX',
       'ECOSPORT', 'FIESTA VAN', 'FOCUS', 'MONDEO', 'MUSTANG', 'CIVIC',
       'H-1', 'H350', 'Q50', 'LS', 'LEAF', 'ASTRA', 'CORSA', '208', '308',
       'KOLEOS', 'SC\xc3\xa9NIC', 'TWIZY', 'ACTYON PROFESIONAL', 'KORANDO',
       'REXTON', 'RODIUS', 'RODIUS MIXTO ADAPTABLE', 'TIVOLI', 'XLV',
       'CELERIO', 'AMAROK', 'CALIFORNIA', 'MULTIVAN', 'XC90'], dtype=object)

## Modelos sin datos de maletero 

In [258]:
df[df['maletero']=='No hay'].modelo.unique()

array(['124 SPIDER', '595', 'A4 ALLROAD', 'A5', 'A6 ALLROAD', 'A7', 'TT',
       'SERIE 2', 'SERIE 3', 'SERIE 4', 'SERIE 5', 'SERIE 6', 'PANDA VAN',
       'TIPO', 'FIESTA VAN', 'KA', 'RANGER', 'H-1', 'H350', 'F-TYPE',
       'CEED', 'CX-3', 'MAZDA2', 'MAZDA3', 'MAZDA6', 'MX-5', 'AMG GT',
       'CLASE B', 'CLASE C', 'CLASE CLA', 'CLASE CLS', 'CLASE E',
       'CLASE GLA', 'CLASE GLC', 'CLASE GLE', 'CLASE GLS', 'CLASE SL',
       'CLASE SLC', 'CLASE V', 'MINI', 'L200', 'MONTERO', '4-4', 'AERO',
       'PLUS 4', 'PLUS 8', 'ROADSTER', 'NAVARA', 'NOTE', 'CORSA',
       'CORSA VAN', 'INSIGNIA', 'INSIGNIA COUNTRY TOURER', 'MOKKA',
       '208 XAD', '911', 'BOXSTER', 'CAYMAN', 'CLIO SOCIET\xc3\xa9',
       'ZOE SOCIET\xc3\xa9', 'LE\xc3\xb3N', 'LE\xc3\xb3N X-PERIENCE',
       'SCOUT', 'ACTYON PROFESIONAL', 'KORANDO', 'REXTON', 'RODIUS',
       'RODIUS MIXTO ADAPTABLE', 'TIVOLI', 'BRZ', 'FORESTER', 'OUTBACK',
       'WRX STI', 'SX4 S-CROSS', 'HILUX', 'LAND CRUISER', 'RAV-4',
       'AMAROK', '

In [323]:
maleteros[maleteros['marca']=='MAZDA'].MODELOS

220               MAZDA 2
221               MAZDA 3
222    MAZDA 3 SPORTSEDAN
223               MAZDA 6
224         MAZDA 6 WAGON
225            MAZDA CX-5
Name: MODELOS, dtype: object

In [319]:
%whos

Variable            Type         Data/Info
------------------------------------------
borrar_citroen      list         n=7
borrar_dacia        list         n=3
borrar_fiat         list         n=6
borrar_ford         list         n=7
borrar_mercedes     list         n=3
borrar_nissan       list         n=5
borrar_opel         list         n=3
borrar_peugeot      list         n=6
borrar_renault      list         n=3
borrar_seat         list         n=1
borrar_toyota       list         n=3
borrar_volkswagen   list         n=5
cols_to_drop        list         n=11
datos               DataFrame           marca      modelo <...>n[5932 rows x 35 columns]
df                  DataFrame           marca      modelo <...>n[4670 rows x 24 columns]
df1                 DataFrame                        marca<...>n[5932 rows x 34 columns]
f                   function     <function f at 0x112812140>
maleteros           DataFrame                marca  malete<...>n\n[427 rows x 3 columns]
mod            

In [436]:
data = [140, 185, 505, 465, 530, 535, 292, 390, 480, 445, 530, 460, 440, 270, 310, 380, 350, 280, 364, 480, 130, 175, 488, 480, 470, 520, 540, 421, 550, 690, 680, 345, 225, 278, 860, 285, 490, 560, 356, 145, 280, 425, 380, 587, 610, 486, 680, 1975, 423, 243, 505, 512, 460, 430, 621, 547, 560]
s = ['124 SPIDER', '595', 'A4 ALLROAD', 'A5', 'A6 ALLROAD', 'A7', 'TT', 'SERIE 2', 'SERIE 3', 'SERIE 4', 'SERIE 5', 'SERIE 6', 'TIPO', 'KA','F-TYPE', 'CEED', 'CX-3', 'MAZDA2', 'MAZDA3', 'MAZDA6', 'MX-5', 'AMG GT', 'CLASE B', 'CLASE C', 'CLASE CLA', 'CLASE CLS', 'CLASE E', 'CLASE GLA', 'CLASE GLC', 'CLASE GLE', 'CLASE GLS', 'CLASE SL', 'CLASE SLC', 'MINI','MONTERO', 'CORSA','INSIGNIA', 'INSIGNIA COUNTRY TOURER', 'MOKKA', '911', 'BOXSTER', 'CAYMAN', 'LE\xc3\xb3N', 'LE\xc3\xb3N X-PERIENCE', 'SCOUT', 'KORANDO', 'REXTON', 'RODIUS', 'TIVOLI', 'BRZ', 'FORESTER', 'OUTBACK', 'WRX STI', 'SX4 S-CROSS', 'LAND CRUISER', 'RAV-4', 'V90 CROSS COUNTRY']

In [437]:
maleteros_faltan = pd.Series(data, index=s)

In [438]:
maleteros_faltan

124 SPIDER                  140
595                         185
A4 ALLROAD                  505
A5                          465
A6 ALLROAD                  530
A7                          535
TT                          292
SERIE 2                     390
SERIE 3                     480
SERIE 4                     445
SERIE 5                     530
SERIE 6                     460
TIPO                        440
KA                          270
F-TYPE                      310
CEED                        380
CX-3                        350
MAZDA2                      280
MAZDA3                      364
MAZDA6                      480
MX-5                        130
AMG GT                      175
CLASE B                     488
CLASE C                     480
CLASE CLA                   470
CLASE CLS                   520
CLASE E                     540
CLASE GLA                   421
CLASE GLC                   550
CLASE GLE                   690
CLASE GLS                   680
CLASE SL

In [338]:
def g(row):
    if row["MODELOS"] not in maleteros_lista:
        return 'No hay'
    else:
        return maleteros_2[row["MODELOS"]]

In [332]:
maleteros_lista = maleteros_2.maletero

In [330]:
maleteros_2.head()

,maletero,modelo
0,140,124 SPIDER
1,185,595
2,505,A4 ALLROAD
3,465,A5
4,530,A6 ALLROAD


In [339]:
df["maletero2"] = datos.apply(g, axis=1)

In [342]:
df[df['maletero2']=='No hay']

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,cilindrada,capacidad deposito,max_vel,0_a_100,rpm,par,estrellas NCAP,carroceria,MODELOS,maletero,maletero2
0,ABARTH,124 SPIDER,Abarth 1.4 170,35093,2,Gasolina,trasera,170,Manual,6,148,703.1,6.4,24,1.368,45.0,232,6.8,5000,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,No hay
1,ABARTH,124 SPIDER,Abarth 1.4 Scorpione 170,36511,2,Gasolina,trasera,170,Manual,6,148,703.1,6.4,24,1.368,45.0,232,6.8,5000,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,No hay
2,ABARTH,124 SPIDER,Abarth 1.4 Aut. 170,37961,2,Gasolina,trasera,170,Automática secuencial,6,153,681.8,6.6,24,1.368,45.0,229,6.9,5000,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,No hay
3,ABARTH,124 SPIDER,Abarth 1.4 Scorpione Aut. 170,38511,2,Gasolina,trasera,170,Automática secuencial,6,153,681.8,6.6,24,1.368,45.0,229,6.9,5000,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,No hay
4,ABARTH,595,Abarth 1.4T-Jet 145,15670,3,Gasolina,delantera,145,Manual,5,139,583.3,6.0,0,1.368,35.0,205,7.9,5500,206.0,-,3660/1627/1480,ABARTH 595,No hay,No hay
5,ABARTH,595,Abarth C 1.4T-Jet 145,17316,2,Gasolina,delantera,145,Manual,5,139,583.3,6.0,0,1.368,35.0,205,7.9,5500,206.0,-,3660/1627/1480,ABARTH 595,No hay,No hay
6,ABARTH,595,Abarth 1.4T-Jet Turismo 165,20497,3,Gasolina,delantera,165,Manual,5,139,583.3,6.0,0,1.368,35.0,210,7.4,5500,230.0,-,3660/1627/1485,ABARTH 595,No hay,No hay
7,ABARTH,595,Abarth 1.4T-Jet 145 Aut.,21086,3,Gasolina,delantera,145,Automática secuencial,5,139,583.3,6.0,0,1.368,35.0,205,7.9,5500,206.0,-,3660/1627/1480,ABARTH 595,No hay,No hay
8,ABARTH,595,Abarth 1.4T-Jet Pista 160,21861,3,Gasolina,delantera,160,Manual,5,139,583.3,6.0,0,1.368,35.0,210,7.4,5500,230.0,-,3660/1627/1485,ABARTH 595,No hay,No hay
9,ABARTH,595,Abarth C 1.4T-Jet Turismo 165,22208,2,Gasolina,delantera,165,Manual,5,139,583.3,6.0,0,1.368,35.0,210,7.4,5500,230.0,-,3660/1627/1485,ABARTH 595,No hay,No hay


In [360]:
def seleccion_maleteros(row):
    if row['maletero']!='No hay':
        return row['maletero']
    elif(row['maletero2']!='No hay'):
        return row['maletero2']
    else:
        return 'No'

In [363]:
df["maleteros"] = datos.apply(seleccion_maleteros, axis=1)

In [366]:
df.head(300)

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,cilindrada,capacidad deposito,max_vel,0_a_100,rpm,par,estrellas NCAP,carroceria,MODELOS,maletero,maletero2,maleteros
0,ABARTH,124 SPIDER,Abarth 1.4 170,35093,2,Gasolina,trasera,170,Manual,6,148,703.1,6.4,24,1.368,45.0,232,6.8,5000,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,No hay,NaN
1,ABARTH,124 SPIDER,Abarth 1.4 Scorpione 170,36511,2,Gasolina,trasera,170,Manual,6,148,703.1,6.4,24,1.368,45.0,232,6.8,5000,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,No hay,NaN
2,ABARTH,124 SPIDER,Abarth 1.4 Aut. 170,37961,2,Gasolina,trasera,170,Automática secuencial,6,153,681.8,6.6,24,1.368,45.0,229,6.9,5000,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,No hay,NaN
3,ABARTH,124 SPIDER,Abarth 1.4 Scorpione Aut. 170,38511,2,Gasolina,trasera,170,Automática secuencial,6,153,681.8,6.6,24,1.368,45.0,229,6.9,5000,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,No hay,NaN
4,ABARTH,595,Abarth 1.4T-Jet 145,15670,3,Gasolina,delantera,145,Manual,5,139,583.3,6.0,0,1.368,35.0,205,7.9,5500,206.0,-,3660/1627/1480,ABARTH 595,No hay,No hay,NaN
5,ABARTH,595,Abarth C 1.4T-Jet 145,17316,2,Gasolina,delantera,145,Manual,5,139,583.3,6.0,0,1.368,35.0,205,7.9,5500,206.0,-,3660/1627/1480,ABARTH 595,No hay,No hay,NaN
6,ABARTH,595,Abarth 1.4T-Jet Turismo 165,20497,3,Gasolina,delantera,165,Manual,5,139,583.3,6.0,0,1.368,35.0,210,7.4,5500,230.0,-,3660/1627/1485,ABARTH 595,No hay,No hay,NaN
7,ABARTH,595,Abarth 1.4T-Jet 145 Aut.,21086,3,Gasolina,delantera,145,Automática secuencial,5,139,583.3,6.0,0,1.368,35.0,205,7.9,5500,206.0,-,3660/1627/1480,ABARTH 595,No hay,No hay,NaN
8,ABARTH,595,Abarth 1.4T-Jet Pista 160,21861,3,Gasolina,delantera,160,Manual,5,139,583.3,6.0,0,1.368,35.0,210,7.4,5500,230.0,-,3660/1627/1485,ABARTH 595,No hay,No hay,NaN
9,ABARTH,595,Abarth C 1.4T-Jet Turismo 165,22208,2,Gasolina,delantera,165,Manual,5,139,583.3,6.0,0,1.368,35.0,210,7.4,5500,230.0,-,3660/1627/1485,ABARTH 595,No hay,No hay,NaN


### Meter carrocería / segmento del vehículo 

In [367]:
!ls

Construcción Dataframe definitivo.ipynb
Dataframe_limpio.csv
FlaskApp
Limpieza datos publicos.ipynb
README.md
README.txt
Scrapeo-Maleteros.ipynb
Script scraping - Dataframe (Antiguo).ipynb
Script scrapping - Dataframe.ipynb
TFM_scraping.ipynb
datos_con_maletero.csv
datos_crudo
datos_limpios
maleteros
segmentos.csv
segmentos.numbers
total_segmento.csv
variaciones_segmento.csv


In [368]:
!cat segmentos.csv












































































































































































































































































































































































































VOLVO;Berlina;XC70

In [412]:
segmentos = pd.read_csv('segmentos.csv',sep=';')

In [413]:
listado_segmentos = segmentos.Modelo

In [418]:
listado_segmentos.dtypes

dtype('O')

In [414]:
def segmentos_tot(row):
    if row["modelo"] not in listado_segmentos:
        return 'No hay'
    else:
        return segmentos[segmentos[row["modelo"]]].Segmento

In [415]:
df['segmento'] = df.apply(segmentos_tot, axis=1)

In [450]:
df.head(100)

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,cilindrada,capacidad deposito,max_vel,0_a_100,rpm,par,estrellas NCAP,carroceria,MODELOS,segmento
0,ABARTH,124 SPIDER,Abarth 1.4 170,35093,2,Gasolina,trasera,170,Manual,6,148,703.1,6.4,24,1.368,45.0,232,6.8,5000,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay
1,ABARTH,124 SPIDER,Abarth 1.4 Scorpione 170,36511,2,Gasolina,trasera,170,Manual,6,148,703.1,6.4,24,1.368,45.0,232,6.8,5000,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay
2,ABARTH,124 SPIDER,Abarth 1.4 Aut. 170,37961,2,Gasolina,trasera,170,Automática secuencial,6,153,681.8,6.6,24,1.368,45.0,229,6.9,5000,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay
3,ABARTH,124 SPIDER,Abarth 1.4 Scorpione Aut. 170,38511,2,Gasolina,trasera,170,Automática secuencial,6,153,681.8,6.6,24,1.368,45.0,229,6.9,5000,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay
4,ABARTH,595,Abarth 1.4T-Jet 145,15670,3,Gasolina,delantera,145,Manual,5,139,583.3,6.0,0,1.368,35.0,205,7.9,5500,206.0,-,3660/1627/1480,ABARTH 595,No hay
5,ABARTH,595,Abarth C 1.4T-Jet 145,17316,2,Gasolina,delantera,145,Manual,5,139,583.3,6.0,0,1.368,35.0,205,7.9,5500,206.0,-,3660/1627/1480,ABARTH 595,No hay
6,ABARTH,595,Abarth 1.4T-Jet Turismo 165,20497,3,Gasolina,delantera,165,Manual,5,139,583.3,6.0,0,1.368,35.0,210,7.4,5500,230.0,-,3660/1627/1485,ABARTH 595,No hay
7,ABARTH,595,Abarth 1.4T-Jet 145 Aut.,21086,3,Gasolina,delantera,145,Automática secuencial,5,139,583.3,6.0,0,1.368,35.0,205,7.9,5500,206.0,-,3660/1627/1480,ABARTH 595,No hay
8,ABARTH,595,Abarth 1.4T-Jet Pista 160,21861,3,Gasolina,delantera,160,Manual,5,139,583.3,6.0,0,1.368,35.0,210,7.4,5500,230.0,-,3660/1627/1485,ABARTH 595,No hay
9,ABARTH,595,Abarth C 1.4T-Jet Turismo 165,22208,2,Gasolina,delantera,165,Manual,5,139,583.3,6.0,0,1.368,35.0,210,7.4,5500,230.0,-,3660/1627/1485,ABARTH 595,No hay


In [449]:
df.drop(['maletero','maletero2','maleteros'], axis=1, inplace=True)

In [490]:
def seleccion_maleteros(row):
    if row['modelo'] in s:
        return maleteros_faltan[row['modelo']]
    else:
        return '0'

In [488]:
df["maleteros"] = df.apply(seleccion_maleteros, axis=1)

124 SPIDER
1
124 SPIDER
140
124 SPIDER
1
124 SPIDER
140
124 SPIDER
1
124 SPIDER
140
124 SPIDER
1
124 SPIDER
140
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
595
1
595
185
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A1
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A3
0
A4
0
A4
0
A4
0
A4
0
A4
0
A4
0
A4
0
A4
0
A4
0
A4
0
A4
0
A4
0
A4
0
A4
0
A4
0
A4
0
A4
0
A4
0
A4
0
A

In [494]:
def f(row):
  if row["MODELOS"] not in modelos_maleteros:
    print(row["MODELOS"])
    return '0'
  else:
    return maleteros[maleteros['MODELOS']==row["MODELOS"]].maletero.values[0]

In [495]:
df["maleteros2"] = df.apply(f, axis=1)

ABARTH 124 SPIDER
ABARTH 124 SPIDER
ABARTH 124 SPIDER
ABARTH 124 SPIDER
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 124 SPIDER
ABARTH 124 SPIDER
ABARTH 124 SPIDER
ABARTH 124 SPIDER
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595
ABARTH 595


TypeError: ('list indices must be integers, not str', u'occurred at index 32')

In [478]:
maleteros_faltan['A4 ALLROAD']

505

In [489]:
df.head(33)

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,cilindrada,capacidad deposito,max_vel,0_a_100,rpm,par,estrellas NCAP,carroceria,MODELOS,segmento,maleteros
0,ABARTH,124 SPIDER,Abarth 1.4 170,35093,2,Gasolina,trasera,170,Manual,6,148,703.1,6.4,24,1.368,45.0,232,6.8,5000,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,140
1,ABARTH,124 SPIDER,Abarth 1.4 Scorpione 170,36511,2,Gasolina,trasera,170,Manual,6,148,703.1,6.4,24,1.368,45.0,232,6.8,5000,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,140
2,ABARTH,124 SPIDER,Abarth 1.4 Aut. 170,37961,2,Gasolina,trasera,170,Automática secuencial,6,153,681.8,6.6,24,1.368,45.0,229,6.9,5000,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,140
3,ABARTH,124 SPIDER,Abarth 1.4 Scorpione Aut. 170,38511,2,Gasolina,trasera,170,Automática secuencial,6,153,681.8,6.6,24,1.368,45.0,229,6.9,5000,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,140
4,ABARTH,595,Abarth 1.4T-Jet 145,15670,3,Gasolina,delantera,145,Manual,5,139,583.3,6.0,0,1.368,35.0,205,7.9,5500,206.0,-,3660/1627/1480,ABARTH 595,No hay,185
5,ABARTH,595,Abarth C 1.4T-Jet 145,17316,2,Gasolina,delantera,145,Manual,5,139,583.3,6.0,0,1.368,35.0,205,7.9,5500,206.0,-,3660/1627/1480,ABARTH 595,No hay,185
6,ABARTH,595,Abarth 1.4T-Jet Turismo 165,20497,3,Gasolina,delantera,165,Manual,5,139,583.3,6.0,0,1.368,35.0,210,7.4,5500,230.0,-,3660/1627/1485,ABARTH 595,No hay,185
7,ABARTH,595,Abarth 1.4T-Jet 145 Aut.,21086,3,Gasolina,delantera,145,Automática secuencial,5,139,583.3,6.0,0,1.368,35.0,205,7.9,5500,206.0,-,3660/1627/1480,ABARTH 595,No hay,185
8,ABARTH,595,Abarth 1.4T-Jet Pista 160,21861,3,Gasolina,delantera,160,Manual,5,139,583.3,6.0,0,1.368,35.0,210,7.4,5500,230.0,-,3660/1627/1485,ABARTH 595,No hay,185
9,ABARTH,595,Abarth C 1.4T-Jet Turismo 165,22208,2,Gasolina,delantera,165,Manual,5,139,583.3,6.0,0,1.368,35.0,210,7.4,5500,230.0,-,3660/1627/1485,ABARTH 595,No hay,185


In [434]:
df[df.maleteros.isnull()].modelo.unique()

array(['124 SPIDER', '595', 'A4 ALLROAD', 'A5', 'A6 ALLROAD', 'A7', 'TT',
       'SERIE 2', 'SERIE 3', 'SERIE 4', 'SERIE 5', 'SERIE 6', 'PANDA VAN',
       'TIPO', 'FIESTA VAN', 'KA', 'RANGER', 'H-1', 'H350', 'F-TYPE',
       'CEED', 'CX-3', 'MAZDA2', 'MAZDA3', 'MAZDA6', 'MX-5', 'AMG GT',
       'CLASE B', 'CLASE C', 'CLASE CLA', 'CLASE CLS', 'CLASE E',
       'CLASE GLA', 'CLASE GLC', 'CLASE GLE', 'CLASE GLS', 'CLASE SL',
       'CLASE SLC', 'CLASE V', 'MINI', 'L200', 'MONTERO', '4-4', 'AERO',
       'PLUS 4', 'PLUS 8', 'ROADSTER', 'NAVARA', 'NOTE', 'CORSA',
       'CORSA VAN', 'INSIGNIA', 'INSIGNIA COUNTRY TOURER', 'MOKKA',
       '208 XAD', '911', 'BOXSTER', 'CAYMAN', 'CLIO SOCIET\xc3\xa9',
       'ZOE SOCIET\xc3\xa9', 'LE\xc3\xb3N', 'LE\xc3\xb3N X-PERIENCE',
       'SCOUT', 'ACTYON PROFESIONAL', 'KORANDO', 'REXTON', 'RODIUS',
       'RODIUS MIXTO ADAPTABLE', 'TIVOLI', 'BRZ', 'FORESTER', 'OUTBACK',
       'WRX STI', 'SX4 S-CROSS', 'HILUX', 'LAND CRUISER', 'RAV-4',
       'AMAROK', '

In [496]:
modelos_maleteros

['ALFA ROMEO MITO',
 'ALFA ROMEO 4C',
 'ALFA ROMEO GIULIETTA',
 'ALFA ROMEO GIULIA',
 'ALFA ROMEO STELVIO',
 'ASTON MARTIN V12 VANTAGE S',
 'ASTON MARTIN DB11',
 'ASTON MARTIN RAPIDE S',
 'ASTON MARTIN VANQUISH COUPE',
 'ASTON MARTIN VANQUISH VOLANTE',
 'AUDI A1',
 'AUDI A1 SPORTBACK',
 'AUDI S1',
 'AUDI A3',
 'AUDI A3 SPORTBACK',
 'AUDI A3 CABRIO',
 'AUDI A3 SEDAN',
 'AUDI S3',
 'AUDI A4',
 'AUDI A4 AVANT',
 'AUDI A4 ALLROAD QUATTRO',
 'AUDI S4',
 'AUDI A5 COUPE',
 'AUDI A5 SPORTBACK',
 'AUDI S5',
 'AUDI A6',
 'AUDI A6 AVANT',
 'AUDI A7 SPORTBACK',
 'AUDI A8',
 'AUDI A8 L',
 'AUDI TT COUPE',
 'AUDI R8',
 'AUDI Q2',
 'AUDI Q3',
 'AUDI Q5',
 'AUDI Q7',
 'BENTLEY CONTINENTAL GT',
 'BENTLEY CONTINENTAL GT CONVERTIBLE',
 'BENTLEY BENTAYGA',
 'BENTLEY FLYING SPUR',
 'BENTLEY MULSANNE',
 'BMW SERIE 1',
 'BMW I3',
 'BMW SERIE 2 COUPE',
 'BMW SERIE 2 CABRIO',
 'BMW SERIE 2 ACTIVE TOURER',
 'BMW SERIE 2 GRAN TOURER',
 'BMW M2 COUPE',
 'BMW SERIE 3 BERLINA',
 'BMW SERIE 3 TOURING',
 'BMW M3 BERL

In [497]:
maleteros_faltan

124 SPIDER                  140
595                         185
A4 ALLROAD                  505
A5                          465
A6 ALLROAD                  530
A7                          535
TT                          292
SERIE 2                     390
SERIE 3                     480
SERIE 4                     445
SERIE 5                     530
SERIE 6                     460
TIPO                        440
KA                          270
F-TYPE                      310
CEED                        380
CX-3                        350
MAZDA2                      280
MAZDA3                      364
MAZDA6                      480
MX-5                        130
AMG GT                      175
CLASE B                     488
CLASE C                     480
CLASE CLA                   470
CLASE CLS                   520
CLASE E                     540
CLASE GLA                   421
CLASE GLC                   550
CLASE GLE                   690
CLASE GLS                   680
CLASE SL

In [502]:
maleteros

[140,
 185,
 505,
 465,
 530,
 535,
 292,
 390,
 480,
 445,
 530,
 460,
 440,
 270,
 310,
 380,
 350,
 280,
 364,
 480,
 130,
 175,
 488,
 480,
 470,
 520,
 540,
 421,
 550,
 690,
 680,
 345,
 225,
 278,
 860,
 285,
 490,
 560,
 356,
 145,
 280,
 425,
 380,
 587,
 610,
 486,
 680,
 1975,
 423,
 243,
 505,
 512,
 460,
 430,
 621,
 547,
 560]

In [512]:
maleteros

,Marca,Modelo,CapacidadMaletero
0,ALFA ROMEO,Alfa Romeo MiTo,270
1,ALFA ROMEO,Alfa Romeo 4C,110
2,ALFA ROMEO,Alfa Romeo Giulietta,350
3,ALFA ROMEO,Alfa Romeo Giulia,480
4,ALFA ROMEO,Nuevo Alfa Romeo Stelvio 2017,525
5,ASTON_MARTIN,Aston Martin V12 Vantage S,300
6,ASTON_MARTIN,Nuevo Aston Martin DB11 2017,270
7,ASTON_MARTIN,Aston Martin Rapide S,317
8,ASTON_MARTIN,Aston Martin Vanquish Coupe,368
9,ASTON_MARTIN,Aston Martin Vanquish Volante,279


In [521]:
maleteros.head()

,marca,maletero,MODELOS
0,ALFA ROMEO,270,ALFA ROMEO MITO
1,ALFA ROMEO,110,ALFA ROMEO 4C
2,ALFA ROMEO,350,ALFA ROMEO GIULIETTA
3,ALFA ROMEO,480,ALFA ROMEO GIULIA
4,ALFA ROMEO,525,ALFA ROMEO STELVIO


In [533]:
maleteros.drop('marca', axis=1, inplace=True)

In [538]:
maleteros = maleteros.reindex(columns=['MODELOS','maletero'])

In [539]:
maleteros.head()

,MODELOS,maletero
0,ALFA ROMEO MITO,270
1,ALFA ROMEO 4C,110
2,ALFA ROMEO GIULIETTA,350
3,ALFA ROMEO GIULIA,480
4,ALFA ROMEO STELVIO,525


In [583]:
maleteros2 = pd.DataFrame({'modelo':s,'maletero_2':data})

In [584]:
maleteros2.head()

,maletero_2,modelo
0,140,124 SPIDER
1,185,595
2,505,A4 ALLROAD
3,465,A5
4,530,A6 ALLROAD


In [565]:
concat = pd.concat([maleteros,maleteros2])

In [567]:
concat.count()

MODELOS     484
maletero    484
dtype: int64

In [581]:
tot1 = pd.merge(df,maleteros,how='outer')

In [585]:
tot2 = pd.merge(tot1,maleteros2,how='outer',left_on='modelo', right_on='modelo')

In [586]:
tot2.head(100)

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,cilindrada,capacidad deposito,max_vel,0_a_100,rpm,par,estrellas NCAP,carroceria,MODELOS,segmento,maletero,maletero_2
0,ABARTH,124 SPIDER,Abarth 1.4 170,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,232.0,6.8,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,NaN,140.0
1,ABARTH,124 SPIDER,Abarth 1.4 Scorpione 170,36511.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,232.0,6.8,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,NaN,140.0
2,ABARTH,124 SPIDER,Abarth 1.4 Aut. 170,37961.0,2,Gasolina,trasera,170.0,Automática secuencial,6,153.0,681.8,6.6,24.0,1.368,45.0,229.0,6.9,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,NaN,140.0
3,ABARTH,124 SPIDER,Abarth 1.4 Scorpione Aut. 170,38511.0,2,Gasolina,trasera,170.0,Automática secuencial,6,153.0,681.8,6.6,24.0,1.368,45.0,229.0,6.9,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,NaN,140.0
4,FIAT,124 SPIDER,1.4 Multiair 140,23229.0,2,Gasolina,trasera,140.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,215.0,7.5,5000.0,240.0,-,4054/1740/1233,FIAT 124 SPIDER,No hay,140.0,140.0
5,FIAT,124 SPIDER,1.4 Multiair Lusso 140,25109.0,2,Gasolina,trasera,140.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,215.0,7.5,5000.0,240.0,-,4054/1740/1233,FIAT 124 SPIDER,No hay,140.0,140.0
6,FIAT,124 SPIDER,1.4 Multiair Lusso Aut. 140,30260.0,2,Gasolina,trasera,140.0,Automática secuencial,6,148.0,703.1,6.4,24.0,1.368,45.0,214.0,7.6,5000.0,240.0,-,4054/1740/1233,FIAT 124 SPIDER,No hay,140.0,140.0
7,FIAT,124 SPIDER,1.4 Multiair Europa 140,32260.0,2,Gasolina,trasera,140.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,215.0,7.5,5000.0,240.0,-,4054/1740/1233,FIAT 124 SPIDER,No hay,140.0,140.0
8,FIAT,124 SPIDER,1.4 Multiair America 140,32260.0,2,Gasolina,trasera,140.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,215.0,7.5,5000.0,240.0,-,4054/1740/1233,FIAT 124 SPIDER,No hay,140.0,140.0
9,FIAT,124 SPIDER,1.4 Multiair America Aut. 140,34260.0,2,Gasolina,trasera,140.0,Automática secuencial,6,148.0,703.1,6.4,24.0,1.368,45.0,214.0,7.6,5000.0,240.0,-,4054/1740/1233,FIAT 124 SPIDER,No hay,140.0,140.0


In [547]:
df.drop('maleteros', axis=1, inplace=True)

In [604]:
def maleteros(row):
    if(np.isnan(row['maletero'])):  
       return row['maletero_2']
    else:
       return row['maletero']

In [605]:
tot2['maleteroF'] = tot2.apply(maleteros, axis=1)

## A borrar 

In [615]:
borrar_modelos = tot2[np.isnan(tot2.maleteroF)].index

In [613]:
borrar_modelos = ['PANDA VAN', 'FIESTA VAN', 'RANGER', 'H-1', 'H350', 'CLASE V',
       'L200', '4-4', 'AERO', 'PLUS 4', 'PLUS 8', 'ROADSTER', 'NAVARA',
       'NOTE', 'CORSA VAN', '208 XAD', 'CLIO SOCIET\xc3\xa9',
       'ZOE SOCIET\xc3\xa9', 'ACTYON PROFESIONAL',
       'RODIUS MIXTO ADAPTABLE', 'HILUX', 'AMAROK', 'CALIFORNIA',
       'MULTIVAN']

In [616]:
tot2.drop(borrar_modelos, inplace=True)

In [617]:
tot2[np.isnan(tot2.maleteroF)]

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,cilindrada,capacidad deposito,max_vel,0_a_100,rpm,par,estrellas NCAP,carroceria,MODELOS,segmento,maletero,maletero_2,maleteroF


In [618]:
tot2.drop(['maletero','maletero_2'], axis=1, inplace=True)

In [619]:
tot2.head()

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,cilindrada,capacidad deposito,max_vel,0_a_100,rpm,par,estrellas NCAP,carroceria,MODELOS,segmento,maleteroF
0,ABARTH,124 SPIDER,Abarth 1.4 170,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,232.0,6.8,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,140.0
1,ABARTH,124 SPIDER,Abarth 1.4 Scorpione 170,36511.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,232.0,6.8,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,140.0
2,ABARTH,124 SPIDER,Abarth 1.4 Aut. 170,37961.0,2,Gasolina,trasera,170.0,Automática secuencial,6,153.0,681.8,6.6,24.0,1.368,45.0,229.0,6.9,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,140.0
3,ABARTH,124 SPIDER,Abarth 1.4 Scorpione Aut. 170,38511.0,2,Gasolina,trasera,170.0,Automática secuencial,6,153.0,681.8,6.6,24.0,1.368,45.0,229.0,6.9,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,140.0
4,FIAT,124 SPIDER,1.4 Multiair 140,23229.0,2,Gasolina,trasera,140.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,215.0,7.5,5000.0,240.0,-,4054/1740/1233,FIAT 124 SPIDER,No hay,140.0


In [622]:
tot2['maletero'] = tot2['maleteroF'].astype(int)

In [623]:
tot2.drop(['maleteroF'], axis=1, inplace=True)

In [624]:
tot2.head()

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,cilindrada,capacidad deposito,max_vel,0_a_100,rpm,par,estrellas NCAP,carroceria,MODELOS,segmento,maletero
0,ABARTH,124 SPIDER,Abarth 1.4 170,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,232.0,6.8,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,140
1,ABARTH,124 SPIDER,Abarth 1.4 Scorpione 170,36511.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,232.0,6.8,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,140
2,ABARTH,124 SPIDER,Abarth 1.4 Aut. 170,37961.0,2,Gasolina,trasera,170.0,Automática secuencial,6,153.0,681.8,6.6,24.0,1.368,45.0,229.0,6.9,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,140
3,ABARTH,124 SPIDER,Abarth 1.4 Scorpione Aut. 170,38511.0,2,Gasolina,trasera,170.0,Automática secuencial,6,153.0,681.8,6.6,24.0,1.368,45.0,229.0,6.9,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,140
4,FIAT,124 SPIDER,1.4 Multiair 140,23229.0,2,Gasolina,trasera,140.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,215.0,7.5,5000.0,240.0,-,4054/1740/1233,FIAT 124 SPIDER,No hay,140


In [625]:
tot2.marca.unique()

array(['ABARTH', 'FIAT', 'AUDI', 'BMW', 'CITROEN', 'DACIA', 'DS', 'FORD',
       'HONDA', 'HYUNDAI', 'INFINITI', 'JAGUAR', 'JEEP', 'KIA', 'LEXUS',
       'MASERATI', 'MAZDA', 'MERCEDES', 'MINI', 'MITSUBISHI', 'NISSAN',
       'OPEL', 'PEUGEOT', 'PORSCHE', 'RENAULT', 'SEAT', 'SKODA', 'SMART',
       'SSANGYONG', 'SUBARU', 'SUZUKI', 'TESLA', 'TOYOTA', 'VOLKSWAGEN',
       'VOLVO', nan], dtype=object)

In [626]:
marcas = ['ABARTH', 'FIAT', 'AUDI', 'BMW', 'CITROEN', 'DACIA', 'DS', 'FORD',
       'HONDA', 'HYUNDAI', 'INFINITI', 'JAGUAR', 'JEEP', 'KIA', 'LEXUS',
       'MASERATI', 'MAZDA', 'MERCEDES', 'MINI', 'MITSUBISHI', 'NISSAN',
       'OPEL', 'PEUGEOT', 'PORSCHE', 'RENAULT', 'SEAT', 'SKODA', 'SMART',
       'SSANGYONG', 'SUBARU', 'SUZUKI', 'TESLA', 'TOYOTA', 'VOLKSWAGEN',
       'VOLVO']

In [627]:
tipos_marca = [1,1,2,2,1,1,1,1,1,1,2,2,1,1,2,3,1,2,2,1,1,1,1,3,1,1,1,1,1,1,1,3,1,1,2]

In [628]:
marcas_type = pd.DataFrame({'marca':marcas,'tipo_marca':tipos_marca})

In [629]:
marcas_type

,marca,tipo_marca
0,ABARTH,1
1,FIAT,1
2,AUDI,2
3,BMW,2
4,CITROEN,1
5,DACIA,1
6,DS,1
7,FORD,1
8,HONDA,1
9,HYUNDAI,1


In [ ]:
tot2 = pd.merge(tot2,marcas_type,how='outer')

In [631]:
tot2.head()

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,cilindrada,capacidad deposito,max_vel,0_a_100,rpm,par,estrellas NCAP,carroceria,MODELOS,segmento,maletero,tipo_marca
0,ABARTH,124 SPIDER,Abarth 1.4 170,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,232.0,6.8,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,140,1.0
1,ABARTH,124 SPIDER,Abarth 1.4 Scorpione 170,36511.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,232.0,6.8,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,140,1.0
2,ABARTH,124 SPIDER,Abarth 1.4 Aut. 170,37961.0,2,Gasolina,trasera,170.0,Automática secuencial,6,153.0,681.8,6.6,24.0,1.368,45.0,229.0,6.9,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,140,1.0
3,ABARTH,124 SPIDER,Abarth 1.4 Scorpione Aut. 170,38511.0,2,Gasolina,trasera,170.0,Automática secuencial,6,153.0,681.8,6.6,24.0,1.368,45.0,229.0,6.9,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,No hay,140,1.0
4,ABARTH,595,Abarth 1.4T-Jet 145,15670.0,3,Gasolina,delantera,145.0,Manual,5,139.0,583.3,6.0,0.0,1.368,35.0,205.0,7.9,5500.0,206.0,-,3660/1627/1480,ABARTH 595,No hay,185,1.0


In [637]:
segmentos = segmentos.reindex(columns=['Marca','Modelo','Segmento'])

In [640]:
segmentos.drop('Marca', axis=1, inplace=True)

In [639]:
tot2.drop('segmento',axis=1,inplace=True)

In [641]:
tot2 = pd.merge(tot2,segmentos,how='outer',left_on='modelo', right_on='Modelo')

In [642]:
tot2.head()

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,cilindrada,capacidad deposito,max_vel,0_a_100,rpm,par,estrellas NCAP,carroceria,MODELOS,maletero,tipo_marca,Modelo,Segmento
0,ABARTH,124 SPIDER,Abarth 1.4 170,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,232.0,6.8,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,124 SPIDER,Deportivo bajo
1,ABARTH,124 SPIDER,Abarth 1.4 170,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,232.0,6.8,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,124 SPIDER,Deportivo bajo
2,ABARTH,124 SPIDER,Abarth 1.4 Scorpione 170,36511.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,232.0,6.8,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,124 SPIDER,Deportivo bajo
3,ABARTH,124 SPIDER,Abarth 1.4 Scorpione 170,36511.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,232.0,6.8,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,124 SPIDER,Deportivo bajo
4,ABARTH,124 SPIDER,Abarth 1.4 Aut. 170,37961.0,2,Gasolina,trasera,170.0,Automática secuencial,6,153.0,681.8,6.6,24.0,1.368,45.0,229.0,6.9,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,124 SPIDER,Deportivo bajo


In [643]:
tot2.Segmento.unique()

array(['Deportivo bajo', nan, 'Superutilitario', 'Compacto',
       'Utilitario/Polivalente', 'Familiar', 'Berlina', 'Gran lujo',
       'TTE Peque\xc3\xb1o', 'TTE Compacto', 'TTE Familiar',
       'TTE Superior', 'Deportivo alto', 'Monovolumen compacto',
       'Monovolumen polivalente', 'Monovolumen familiar'], dtype=object)

In [649]:
tot2[tot2.Segmento.isnull()].modelo.unique()

array(['595', '500L', '500X', 'PUNTO', 'A4 ALLROAD', 'A6 ALLROAD', 'I3',
       'I8', 'C-ELYS\xc3\xa9E', 'C3 AIRCROSS', 'DS4 CROSSBACK', 'C-MAX',
       'CR-V', 'HR-V', 'I10', 'I20', 'I20 ACTIVE', 'I30', 'I40', 'IX20',
       'Q60', 'E-PACE', 'F-TYPE CONVERTIBLE', 'LC', 'GRANTURISMO',
       'MAZDA2', 'MAZDA3', 'MAZDA6', 'AMG GT', 'COUNTRYMAN', 'I-MIEV',
       '370Z', 'CROSSLAND X', 'GRANDLAND X', 'INSIGNIA COUNTRY TOURER',
       '508 RXH', 'ION', 'KOLEOS', 'M\xc3\xa9GANE', 'SC\xc3\xa9NIC',
       'TWIZY', 'LE\xc3\xb3N', 'LE\xc3\xb3N X-PERIENCE', 'MII', 'KODIAQ',
       'SCOUT', 'BRZ', 'WRX STI', 'IGNIS', 'GT86', 'LAND CRUISER',
       'ARTEON', 'GOLF ALLTRACK', 'PASSAT ALLTRACK', 'UP',
       'V90 CROSS COUNTRY', 'XC60', 'XC90', nan], dtype=object)

In [671]:
segmentos.head(150)

,Modelo,Segmento
0,NaN,NaN
1,124 SPIDER,Deportivo bajo
2,500,Superutilitario
3,4C,Deportivo bajo
4,GIULIA,Familiar
5,GIULIETTA,Compacto
6,MiTo,Utilitario/Polivalente
7,DB11,Deportivo alto
8,DB9,Deportivo alto
9,RAPIDE,Gran lujo


In [653]:
mod = ['595', '500L', '500X', 'PUNTO', 'A4 ALLROAD', 'A6 ALLROAD', 'I3',
       'I8', 'C-ELYS\xc3\xa9E', 'C3 AIRCROSS', 'DS4 CROSSBACK', 'C-MAX',
       'CR-V', 'HR-V', 'I10', 'I20', 'I20 ACTIVE', 'I30', 'I40', 'IX20',
       'Q60', 'E-PACE', 'F-TYPE CONVERTIBLE', 'LC', 'GRANTURISMO',
       'MAZDA2', 'MAZDA3', 'MAZDA6', 'AMG GT', 'COUNTRYMAN', 'I-MIEV',
       '370Z', 'CROSSLAND X', 'GRANDLAND X', 'INSIGNIA COUNTRY TOURER',
       '508 RXH', 'ION', 'KOLEOS', 'M\xc3\xa9GANE', 'SC\xc3\xa9NIC',
       'TWIZY', 'LE\xc3\xb3N', 'LE\xc3\xb3N X-PERIENCE', 'MII', 'KODIAQ',
       'SCOUT', 'BRZ', 'WRX STI', 'IGNIS', 'GT86', 'LAND CRUISER',
       'ARTEON', 'GOLF ALLTRACK', 'PASSAT ALLTRACK', 'UP',
       'V90 CROSS COUNTRY', 'XC60', 'XC90']

In [701]:
segmentos[segmentos['Modelo']=='CIVIC']

,Modelo,Segmento
116,CIVIC,Compacto


In [704]:
seg = ['Superutilitario', 'Monovolumen polivalente', 'TTE Pequeño', 'Utilitario/Polivalente', 'Familiar', 'Familiar', 'Monovolumen polivalente',
       'Deportivo alto', 'Berlina', 'Monovolumen polivalente', 'Compacto', 'Monovolumen compacto',
       'TTE Familiar', 'TTE Compacto', 'Superutilitario', 'Utilitario/Polivalente', 'Utilitario/Polivalente', 'Compacto', 'Familiar', 'Monovolumen polivalente',
       'Deportivo alto', 'TTE Compacto', 'Deportivo alto', 'Deportivo alto', 'Gran lujo',
       'Utilitario/Polivalente', 'Compacto', 'Berlina', 'Deportivo alto', 'TTE Peque\xc3\xb1o', 'Superutilitario',
       'Deportivo bajo', 'TTE Pequeño', 'TTE Compacto', 'Familiar',
       'Familiar', 'Superutilitario', 'TTE Familiar', 'Compacto', 'Monovolumen familiar',
       'Superutilitario', 'Compacto', 'Familiar', 'Superutilitario', 'TTE Familiar',
       'Familiar', 'Deportivo bajo', 'Compacto', 'Superutilitario', 'Deportivo bajo', 'TTE Superior',
       'Berlina', 'Familiar', 'Familiar', 'Superutilitario',
       'Familiar', 'TTE Familiar', 'TTE Superior']

In [703]:
segmentos[segmentos['Segmento']=='TTE Superior'].Modelo.unique()

array(['Q7', 'X5', 'X6', 'BENTAYGA', 'HUMMER H3', 'GRAND SANTA FE', 'FX',
       'QX70', 'GRAND CHEROKEE', 'DISCOVERY', 'RANGE ROVER',
       'RANGE ROVER SPORT', 'RX', 'LEVANTE', 'CX-9', 'CLASE G', 'CLASE GL',
       'CLASE GLE', 'CLASE GLS', 'CLASE M', 'MONTERO', 'MURANO II',
       'CAYENNE', 'MODEL X', 'LANDCRUISER', 'TOUAREG', 'XC 90'], dtype=object)

In [684]:
segmentos.Segmento.unique()

array([nan, 'Deportivo bajo', 'Superutilitario', 'Familiar', 'Compacto',
       'Utilitario/Polivalente', 'Deportivo alto', 'Gran lujo', 'Berlina',
       'TTE Peque\xc3\xb1o', 'TTE Compacto', 'TTE Familiar',
       'TTE Superior', 'Monovolumen polivalente', 'Monovolumen compacto',
       'Monovolumen familiar'], dtype=object)

In [706]:
segmentos2 = pd.DataFrame({'Modelo':mod, 'Segmento':seg})

In [707]:
segmentos2

,Modelo,Segmento
0,595,Superutilitario
1,500L,Monovolumen polivalente
2,500X,TTE Pequeño
3,PUNTO,Utilitario/Polivalente
4,A4 ALLROAD,Familiar
5,A6 ALLROAD,Familiar
6,I3,Monovolumen polivalente
7,I8,Deportivo alto
8,C-ELYSéE,Berlina
9,C3 AIRCROSS,Monovolumen polivalente


In [708]:
segmentosT = pd.concat(segmentos,segmentos2)

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [709]:
serie_seg1 = pd.Series({'Modelo':segmentos.Modelo, 'Segmento':segmentos.Segmento})

In [710]:
serie_seg2 = pd.Series({'Modelo':mod, 'Segmento':seg})

In [712]:
serie_tot = pd.concat([serie_seg1,serie_seg2])

In [714]:
serie_tot.head()

Modelo      0                    NaN
1             124 SPI...
Segmento    0                          NaN
1              ...
Modelo      [595, 500L, 500X, PUNTO, A4 ALLROAD, A6 ALLROA...
Segmento    [Superutilitario, Monovolumen polivalente, TTE...
dtype: object

In [715]:
df_tot = pd.concat([segmentos,segmentos2], ignore_index=True)

In [718]:
df_tot.tail(100)

,Modelo,Segmento
353,AVENSIS,Familiar
354,AYGO,Superutilitario
355,C-HR,TTE Compacto
356,GT 86,Deportivo bajo
357,HIGHLANDER,TTE Familiar
358,IQ,Superutilitario
359,LANDCRUISER,TTE Superior
360,PRIUS,Compacto
361,PRIUS +,Monovolumen compacto
362,PRIUS PLUG-IN,Compacto


In [722]:
tot2.drop('Segmento', axis=1, inplace=True)

In [723]:
tot2 = pd.merge(tot2,df_tot,how='outer',left_on='modelo', right_on='Modelo')

In [731]:
tot2.modelo

0       124 SPIDER
1       124 SPIDER
2       124 SPIDER
3       124 SPIDER
4       124 SPIDER
5       124 SPIDER
6       124 SPIDER
7       124 SPIDER
8       124 SPIDER
9       124 SPIDER
10      124 SPIDER
11      124 SPIDER
12      124 SPIDER
13      124 SPIDER
14      124 SPIDER
15      124 SPIDER
16      124 SPIDER
17      124 SPIDER
18      124 SPIDER
19      124 SPIDER
20      124 SPIDER
21      124 SPIDER
22      124 SPIDER
23      124 SPIDER
24      124 SPIDER
25      124 SPIDER
26      124 SPIDER
27      124 SPIDER
28      124 SPIDER
29      124 SPIDER
           ...    
6772           NaN
6773           NaN
6774           NaN
6775           NaN
6776           NaN
6777           NaN
6778           NaN
6779           NaN
6780           NaN
6781           NaN
6782           NaN
6783           NaN
6784           NaN
6785           NaN
6786           NaN
6787           NaN
6788           NaN
6789           NaN
6790           NaN
6791           NaN
6792           NaN
6793        

In [735]:
rows_to_drop = tot2[tot2.modelo.isnull()].index

In [736]:
tot2.drop(rows_to_drop, inplace=True)

In [746]:
tot2.Modelo_x.unique()

array(['124 SPIDER', nan, '500', 'PANDA', 'TIPO', 'A1', 'A3', 'A4', 'A5',
       'A6', 'A7', 'A8', 'Q2', 'Q3', 'Q5', 'Q7', 'R8', 'TT', 'SERIE 1',
       'SERIE 2', 'SERIE 3', 'SERIE 4', 'SERIE 5', 'SERIE 6', 'SERIE 7',
       'X1', 'X3', 'X4', 'X5', 'X6', 'C-ZERO', 'C1', 'C3', 'C4',
       'C4 AIRCROSS', 'C4 CACTUS', 'E-MEHARI', 'DUSTER', 'LOGAN',
       'SANDERO', 'DS3', 'DS4', 'DS5', 'B-MAX', 'ECOSPORT', 'EDGE',
       'FIESTA', 'FOCUS', 'GALAXY', 'KA', 'KUGA', 'MONDEO', 'MUSTANG',
       'S-MAX', 'CIVIC', 'JAZZ', 'ELANTRA', 'GENESIS', 'IONIQ', 'SANTA FE',
       'TUCSON', 'Q30', 'Q50', 'Q70', 'QX30', 'QX70', 'F-PACE', 'F-TYPE',
       'XE', 'XF', 'XJ', 'CHEROKEE', 'COMPASS', 'RENEGADE', 'WRANGLER',
       'CARENS', 'CEED', 'NIRO', 'OPTIMA', 'PICANTO', 'RIO', 'SORENTO',
       'SOUL', 'SPORTAGE', 'VENGA', 'CT', 'GS', 'IS', 'LS', 'NX', 'RC',
       'GHIBLI', 'LEVANTE', 'QUATTROPORTE', 'CX-3', 'CX-5', 'MX-5',
       'CLASE A', 'CLASE B', 'CLASE C', 'CLASE CLA', 'CLASE CLS',
       'CLA

In [738]:
tot2.columns

Index([u'marca', u'modelo', u'version', u'precio', u'puertas', u'combustible',
       u'traccion', u'cv', u'cambio', u'n_marchas', u'co2', u'autonomia',
       u'consumo', u'garantia', u'cilindrada', u'capacidad deposito',
       u'max_vel', u'0_a_100', u'rpm', u'par', u'estrellas NCAP',
       u'carroceria', u'MODELOS', u'maletero', u'tipo_marca', u'Modelo_x',
       u'Modelo_y', u'Segmento'],
      dtype='object')

In [760]:
tot2.describe()

,precio,cv,co2,consumo,garantia,cilindrada,capacidad deposito,max_vel,0_a_100,rpm,par,maletero,tipo_marca
count,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000,6222.00000,6222.000000,6222.000000
mean,33316.500964,166.960302,126.574574,5.190148,29.974928,88.054071,55.165831,199.557216,9.168499,4384.386050,301.88073,424.252009,1.460463
std,29236.397245,91.902802,35.115990,1.597681,12.855359,277.307858,12.044219,47.656161,2.821912,1351.289901,142.78565,139.106287,0.531559
min,6513.000000,60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,112.000000,1.000000
25%,17715.000000,115.000000,107.000000,4.200000,24.000000,1.499000,48.000000,185.000000,7.600000,3750.000000,205.00000,350.000000,1.000000
50%,24616.500000,141.000000,119.000000,4.900000,24.000000,1.968000,55.000000,202.000000,9.400000,4200.000000,270.00000,430.000000,1.000000
75%,36179.250000,190.000000,138.000000,5.700000,36.000000,1.999000,62.000000,225.000000,11.000000,5500.000000,370.75000,505.000000,2.000000
max,291180.000000,700.000000,397.000000,17.000000,84.000000,999.000000,100.000000,350.000000,16.900000,9900.000000,1100.00000,1975.000000,3.000000


In [759]:
tot2[tot2.cv==17]

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,cilindrada,capacidad deposito,max_vel,0_a_100,rpm,par,estrellas NCAP,carroceria,MODELOS,maletero,tipo_marca,Modelo_x,Modelo_y,Segmento


In [756]:
row_twizy = tot2[tot2.cv==17].index

In [757]:
tot2.drop(row_twizy, inplace=True)

In [ ]:
row_twizy = tot2[tot2.cv==17].index

In [ ]:
tot2.drop(row_twizy, inplace=True)

In [797]:
tot2[tot2['0_a_100']==0].version.unique()

array(['Sed\xc3\xa1n 1.4 T-Jet Gasolina-GLP Pop 120',
       'Sed\xc3\xa1n 1.4 T-Jet Gasolina-GLP Easy 120',
       'Sed\xc3\xa1n 1.4 T-Jet Gasolina-GLP Lounge 120',
       '1.4 TFSI Advanced edition 150', '1.4 TFSI S line edition 150',
       '1.4 TFSI 150', 'Avant 1.4 TFSI S line edition 150',
       'Avant 1.4 TFSI Advanced edition 150',
       '1.4 TFSI Design edition 150', '2.0TDI S line edition 190',
       'Avant 1.4 TFSI 150', 'Avant 1.4 TFSI Black line edition 150',
       'Avant 1.4 TFSI Design edition 150', '2.0TDI Advanced edition 190',
       '2.0TDI Design edition 190', '2.0TDI 190', 'M760LiA xDrive',
       '1.0 TCE Ambiance 75', 'MCV 1.0 TCE Ambiance 75', '1.0 Base 75',
       '1.0 Ambiance 75', '1.0 Laureate 75', '1.4 Autogas (GLP) Trend',
       '1.4 Autogas (GLP) Titanium', '1.6 Autogas (GLP) Trend+',
       '1.6 Autogas (GLP) Titanium', '1.5TDCi Trend 95',
       '1.5TDCi Titanium 95', '1.5TDCi Titanium S 95',
       '1.5 Ti-VCT Trend PS', '1.5 Ti-VCT Titanium PS',


In [770]:
_max_vel

cv         277     120.0
283     120.0
291     120.0
376 ...
modelo     277                  TIPO
283                 ...
version    277        Sedán 1.4 T-Jet Gasolina-GLP Pop 1...
dtype: object

In [775]:
modelos_sin = ['10.1',
       '10.1',
       '10.1',
       '8.7', '8.7',
       '8.7', '8.7',
       '8.7',
       '8.7', '9.0',
       '8.7', '8.7',
       '8.7', '7.0',
       '7.0', '7.0', '14.2',
       '14.2', '14.2', '14.2',
       '14.2', '12.2',
       '12.2', '12.2',
       '11.5', '11.5',
       '11.5', '11.5',
       '11.0', '11.0',
       '10.0',
       '10.0', '11.0',
       '14.0', '10.0',
       '10.0',
       '10.0', '11.5',
       '14.0', '14.0',
       '14.0', '8.5',
       '7.0',
       '7.0',
       '7.0',
       '7.0',
       '11.0',
       '11.0', '11.5',
       '11.0', '10.0',
       '11.0',
       '11.0',
       '11.0',
       '11.0',
       '14.0', '14.0',
       '10.5', '9.0',
       '10.9', '10.9',
       '10.9', '10.9',
       '10.9', '10.9',
       '10.9', '10.9',
       '1.5 VTEC Turbo Sport', '10.9',
       '1.5 VTEC Turbo Sport CVT', '10.9',
       '10.9', '1.5 VTEC Turbo Sport Plus',
       '10.9', '1.5 VTEC Turbo Prestige',
       '2.0 Mjt Limited 4x4 AD Aut. 170', '500h Business', '500h F Sport',
       '500h Executive', '500h Luxury Art Wood L- White',
       '500h Luxury L-Aniline', '500h Luxury Art Wood',
       '500h Luxury Kiriko', '300h Eco 2WD', '300h Business 2WD',
       '300h Sport Edition 2WD', '300h Executive 4WD', '300h F Sport 4WD',
       '300h F Sport 4WD Panoramico', '300h F Luxury 4WD',
       '300h Executive', '300h F Sport', '300h Luxury', 'Visia 30KWh',
       'Acenta 30KWh', 'Visia+ 30KWh', 'Tekna 30KWh',
       'ST 1.4T S-S Selective 125', 'ST 1.4T S-S Dynamic 125',
       'ST 1.4T S-S Dynamic 150', 'ST 1.6CDTi S-S Business + 136',
       'ST 1.4T S-S Excellence 125', 'ST 1.4T S-S Excellence 150',
       '1.4 GLP Selective 90', 'Grand  1.2 TCe Life 115',
       'Grand  1.2 TCe Intens 130', 'Grand  1.5dCi Life 110',
       'Grand  1.2 TCe Zen 130', 'Grand  1.5dCi Intens 110',
       'Grand  1.5dCi Zen 110', 'Grand  1.5dCi Intens EDC 110',
       'Grand  1.6dCi Zen 130', 'Grand  1.5dCi Zen EDC 110',
       'Grand  1.5dCi Edition One 110', 'Grand  1.6dCi Edition One 130',
       'Grand  1.5dCi Edition One EDC 110',
       'Grand  1.6dCi Edition One 160', '1.0 GA', '1.0 GL', '1.0 GLX',
       '1.0 GLX AGS', 'D4 Plus 190', 'D4 Plus Aut. 190', 'D4 Pro 190',
       'D4 Pro Aut. 190', 'T5 Pro AWD 245 Aut.', 'D4 Momentum 5pl. Aut.',
       'D4 Momentum 7pl. Aut.', 'D4 R-Design Aut.', 'D4 Inscription Aut.',
       'T5 Momentum AWD Aut.', 'T5 R-Design AWD Aut.',
       'T5 Inscription AWD Aut.']

In [776]:
tot2.columns

Index([u'marca', u'modelo', u'version', u'precio', u'puertas', u'combustible',
       u'traccion', u'cv', u'cambio', u'n_marchas', u'co2', u'autonomia',
       u'consumo', u'garantia', u'cilindrada', u'capacidad deposito',
       u'max_vel', u'0_a_100', u'rpm', u'par', u'estrellas NCAP',
       u'carroceria', u'MODELOS', u'maletero', u'tipo_marca', u'Modelo_x',
       u'Modelo_y', u'Segmento'],
      dtype='object')

In [777]:
tot2.head()

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,cilindrada,capacidad deposito,max_vel,0_a_100,rpm,par,estrellas NCAP,carroceria,MODELOS,maletero,tipo_marca,Modelo_x,Modelo_y,Segmento
0,ABARTH,124 SPIDER,Abarth 1.4 170,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,232.0,6.8,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,124 SPIDER,124 SPIDER,Deportivo bajo
1,ABARTH,124 SPIDER,Abarth 1.4 170,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,232.0,6.8,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,124 SPIDER,124 SPIDER,Deportivo bajo
2,ABARTH,124 SPIDER,Abarth 1.4 170,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,232.0,6.8,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,124 SPIDER,124 SPIDER,Deportivo bajo
3,ABARTH,124 SPIDER,Abarth 1.4 170,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,232.0,6.8,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,124 SPIDER,124 SPIDER,Deportivo bajo
4,ABARTH,124 SPIDER,Abarth 1.4 Scorpione 170,36511.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,1.368,45.0,232.0,6.8,5000.0,250.0,-,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,124 SPIDER,124 SPIDER,Deportivo bajo


In [778]:
cols_to_drop = ['cilindrada','max_vel','rpm','estrellas NCAP','Modelo_x','Modelo_y']

In [779]:
tot2.drop(cols_to_drop, axis=1, inplace=True)

In [780]:
tot2.head()

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,capacidad deposito,0_a_100,par,carroceria,MODELOS,maletero,tipo_marca,Segmento
0,ABARTH,124 SPIDER,Abarth 1.4 170,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,6.8,250.0,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo
1,ABARTH,124 SPIDER,Abarth 1.4 170,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,6.8,250.0,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo
2,ABARTH,124 SPIDER,Abarth 1.4 170,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,6.8,250.0,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo
3,ABARTH,124 SPIDER,Abarth 1.4 170,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,6.8,250.0,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo
4,ABARTH,124 SPIDER,Abarth 1.4 Scorpione 170,36511.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,6.8,250.0,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo


In [781]:
tot2.describe()

,precio,cv,co2,consumo,garantia,capacidad deposito,0_a_100,par,maletero,tipo_marca
count,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000,6222.00000,6222.000000,6222.000000
mean,33316.500964,166.960302,126.574574,5.190148,29.974928,55.165831,9.168499,301.88073,424.252009,1.460463
std,29236.397245,91.902802,35.115990,1.597681,12.855359,12.044219,2.821912,142.78565,139.106287,0.531559
min,6513.000000,60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,112.000000,1.000000
25%,17715.000000,115.000000,107.000000,4.200000,24.000000,48.000000,7.600000,205.00000,350.000000,1.000000
50%,24616.500000,141.000000,119.000000,4.900000,24.000000,55.000000,9.400000,270.00000,430.000000,1.000000
75%,36179.250000,190.000000,138.000000,5.700000,36.000000,62.000000,11.000000,370.75000,505.000000,2.000000
max,291180.000000,700.000000,397.000000,17.000000,84.000000,100.000000,16.900000,1100.00000,1975.000000,3.000000


In [794]:
tot2[tot2['par']==0].modelo

491                    A5
492                    A5
493                    A5
494                    A5
811                    I8
1841                LOGAN
1952                B-MAX
1953                B-MAX
1968                C-MAX
1969                C-MAX
1999             ECOSPORT
2000             ECOSPORT
2001             ECOSPORT
2251               MONDEO
2252               MONDEO
2621                  Q70
3031                   GS
3032                   GS
3033                   GS
3034                   GS
3035                   GS
3036                   GS
3037                   GS
3038                   GS
3041                   IS
3042                   IS
3043                   IS
3044                   IS
3045                   IS
3046                   IS
              ...        
5563                 C-HR
5579                RAV-4
5580                RAV-4
5581                RAV-4
5582                RAV-4
5583                RAV-4
5584                RAV-4
5585        

In [798]:
tot2.head()

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,capacidad deposito,0_a_100,par,carroceria,MODELOS,maletero,tipo_marca,Segmento
0,ABARTH,124 SPIDER,Abarth 1.4 170,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,6.8,250.0,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo
1,ABARTH,124 SPIDER,Abarth 1.4 170,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,6.8,250.0,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo
2,ABARTH,124 SPIDER,Abarth 1.4 170,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,6.8,250.0,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo
3,ABARTH,124 SPIDER,Abarth 1.4 170,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,6.8,250.0,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo
4,ABARTH,124 SPIDER,Abarth 1.4 Scorpione 170,36511.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,6.8,250.0,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo


In [803]:
tot2.head()

,marca,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,capacidad deposito,par,carroceria,MODELOS,maletero,tipo_marca,Segmento
0,ABARTH,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,250.0,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo
1,ABARTH,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,250.0,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo
2,ABARTH,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,250.0,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo
3,ABARTH,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,250.0,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo
4,ABARTH,36511.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,250.0,4054/1740/1233,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo


In [804]:
tot2.describe()

,precio,cv,co2,consumo,garantia,capacidad deposito,par,maletero,tipo_marca
count,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000,6222.00000,6222.000000,6222.000000
mean,33316.500964,166.960302,126.574574,5.190148,29.974928,55.165831,301.88073,424.252009,1.460463
std,29236.397245,91.902802,35.115990,1.597681,12.855359,12.044219,142.78565,139.106287,0.531559
min,6513.000000,60.000000,0.000000,0.000000,0.000000,0.000000,0.00000,112.000000,1.000000
25%,17715.000000,115.000000,107.000000,4.200000,24.000000,48.000000,205.00000,350.000000,1.000000
50%,24616.500000,141.000000,119.000000,4.900000,24.000000,55.000000,270.00000,430.000000,1.000000
75%,36179.250000,190.000000,138.000000,5.700000,36.000000,62.000000,370.75000,505.000000,2.000000
max,291180.000000,700.000000,397.000000,17.000000,84.000000,100.000000,1100.00000,1975.000000,3.000000


In [818]:
prueba = tot2.drop(['par'],axis=1)

In [808]:
prueba.dtypes

marca                   object
precio                 float64
puertas               category
combustible           category
traccion              category
cv                     float64
cambio                category
n_marchas             category
co2                    float64
autonomia               object
consumo                float64
garantia               float64
capacidad deposito     float64
carroceria              object
MODELOS                 object
maletero               float64
tipo_marca             float64
Segmento                object
dtype: object

In [813]:
prueba['largo','ancho','alto'] = prueba['carroceria'].str.split('/')

In [836]:
prueba.head()

,marca,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,capacidad deposito,MODELOS,maletero,tipo_marca,Segmento,longitud
0,ABARTH,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo,4054.0
1,ABARTH,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo,4054.0
2,ABARTH,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo,4054.0
3,ABARTH,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo,4054.0
4,ABARTH,36511.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo,4054.0


In [824]:
def longitud(carroceria):
    return carroceria[:4]

In [825]:
prueba['longitud'] = prueba['carroceria'].apply(longitud)

In [827]:
prueba.drop('carroceria',axis=1,inplace=True)

In [835]:
prueba.dtypes

marca                 category
precio                 float64
puertas               category
combustible           category
traccion              category
cv                     float64
cambio                category
n_marchas             category
co2                    float64
autonomia               object
consumo                float64
garantia               float64
capacidad deposito     float64
MODELOS                 object
maletero               float64
tipo_marca             float64
Segmento              category
longitud               float64
dtype: object

In [832]:
prueba['longitud'] = prueba['longitud'].astype(float)

In [833]:
prueba['Segmento'] = prueba['Segmento'].astype('category')

In [834]:
prueba['marca'] = prueba['marca'].astype('category')

In [839]:
prueba.describe()

,precio,cv,co2,consumo,garantia,capacidad deposito,maletero,tipo_marca,longitud
count,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000
mean,33316.500964,166.960302,126.574574,5.190148,29.974928,55.165831,424.252009,1.460463,4440.182739
std,29236.397245,91.902802,35.115990,1.597681,12.855359,12.044219,139.106287,0.531559,389.160430
min,6513.000000,60.000000,0.000000,0.000000,0.000000,0.000000,112.000000,1.000000,2695.000000
25%,17715.000000,115.000000,107.000000,4.200000,24.000000,48.000000,350.000000,1.000000,4253.000000
50%,24616.500000,141.000000,119.000000,4.900000,24.000000,55.000000,430.000000,1.000000,4475.000000
75%,36179.250000,190.000000,138.000000,5.700000,36.000000,62.000000,505.000000,2.000000,4690.000000
max,291180.000000,700.000000,397.000000,17.000000,84.000000,100.000000,1975.000000,3.000000,5462.000000


In [840]:
prueba[prueba['consumo']==17]

,marca,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,capacidad deposito,MODELOS,maletero,tipo_marca,Segmento,longitud
3679,MERCEDES,291180.0,5,Gasolina,Todo terreno tracción total permanente,630.0,Automática secuencial,7,397.0,564.7,17.0,24.0,96.0,MERCEDES CLASE G,480.0,2.0,TTE Superior,4763.0


In [841]:
prueba.traccion.unique()

[trasera, delantera, Todo terreno tracción total permanente, total permanente, Todo terreno tracción total conectable, total conectable]
Categories (6, object): [trasera, delantera, Todo terreno tracción total permanente, total permanente, Todo terreno tracción total conectable, total conectable]

In [861]:
tracciones_totales = ['Todo terreno tracción total permanente', 'total permanente', 'Todo terreno tracción total conectable', 'total conectable']

def traccion(row):
    print row['traccion']
    if(str.strip(row['traccion']) in tracciones_totales):
        return 'total'
    else:
        return row['traccion']
    

In [868]:
prueba['traccion'] = prueba['tipo_traccion']

In [869]:
prueba.drop('tipo_traccion',axis=1,inplace=True)

In [870]:
prueba.head()

,marca,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,capacidad deposito,MODELOS,maletero,tipo_marca,Segmento,longitud
0,ABARTH,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo,4054.0
1,ABARTH,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo,4054.0
2,ABARTH,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo,4054.0
3,ABARTH,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo,4054.0
4,ABARTH,36511.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,ABARTH 124 SPIDER,140.0,1.0,Deportivo bajo,4054.0


In [872]:
prueba['traccion'].unique()

array(['trasera', 'delantera', 'total'], dtype=object)

In [873]:
prueba.describe()

,precio,cv,co2,consumo,garantia,capacidad deposito,maletero,tipo_marca,longitud
count,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000
mean,33316.500964,166.960302,126.574574,5.190148,29.974928,55.165831,424.252009,1.460463,4440.182739
std,29236.397245,91.902802,35.115990,1.597681,12.855359,12.044219,139.106287,0.531559,389.160430
min,6513.000000,60.000000,0.000000,0.000000,0.000000,0.000000,112.000000,1.000000,2695.000000
25%,17715.000000,115.000000,107.000000,4.200000,24.000000,48.000000,350.000000,1.000000,4253.000000
50%,24616.500000,141.000000,119.000000,4.900000,24.000000,55.000000,430.000000,1.000000,4475.000000
75%,36179.250000,190.000000,138.000000,5.700000,36.000000,62.000000,505.000000,2.000000,4690.000000
max,291180.000000,700.000000,397.000000,17.000000,84.000000,100.000000,1975.000000,3.000000,5462.000000


## Preparar el Dataframe para el cálculo de distancias 

In [898]:
from sklearn import preprocessing
import numpy as np

In [874]:
cols_descriptivas = ['marca','MODELOS']

In [875]:
prueba.drop(cols_descriptivas, axis=1,inplace=True)

In [876]:
prueba.head()

,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,capacidad deposito,maletero,tipo_marca,Segmento,longitud
0,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,140.0,1.0,Deportivo bajo,4054.0
1,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,140.0,1.0,Deportivo bajo,4054.0
2,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,140.0,1.0,Deportivo bajo,4054.0
3,35093.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,140.0,1.0,Deportivo bajo,4054.0
4,36511.0,2,Gasolina,trasera,170.0,Manual,6,148.0,703.1,6.4,24.0,45.0,140.0,1.0,Deportivo bajo,4054.0


In [877]:
len(prueba.columns)

16

In [878]:
prueba.cambio.unique()

[Manual, Automática secuencial, Directo. sin caja de cambios, Manual automatizada, Automática continua. secuencial, Automática, Automática continua]
Categories (7, object): [Manual, Automática secuencial, Directo. sin caja de cambios, Manual automatizada, Automática continua. secuencial, Automática, Automática continua]

In [884]:
tot2['cambio'] = tot2['cambio'].apply(str.strip)

In [885]:
tot2[tot2['cambio']=='Manual automatizada']

,marca,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,capacidad deposito,par,carroceria,MODELOS,maletero,tipo_marca,Segmento
1554,CITROEN,9700.0,5,Gasolina,delantera,68.0,Manual automatizada,5,97.0,833.3,4.2,24.0,35.0,95.0,3466/1615/1460,CITROEN C1,196.0,1.0,Superutilitario
1852,DACIA,11301.0,5,Diesel,delantera,90.0,Manual automatizada,6,92.0,1.388.9,3.6,36.0,50.0,220.0,4501/1733/1521,DACIA LOGAN,510.0,1.0,Compacto
1862,DACIA,9189.0,5,Gasolina,delantera,90.0,Manual automatizada,5,110.0,1.020.4,4.9,36.0,50.0,140.0,4069/1733/1519,DACIA SANDERO,320.0,1.0,Utilitario/Polivalente
1867,DACIA,10546.0,5,Diesel,delantera,90.0,Manual automatizada,5,92.0,1.388.9,3.6,36.0,50.0,220.0,4069/1733/1519,DACIA SANDERO,320.0,1.0,Utilitario/Polivalente
1871,DS,13131.0,3,Gasolina,delantera,82.0,Manual automatizada,5,99.0,1.162.8,4.3,24.0,50.0,118.0,3948/1715/1458,DS DS3,285.0,1.0,Utilitario/Polivalente
4467,OPEL,8420.0,5,Gasolina,delantera,75.0,Manual automatizada,5,103.0,711.1,4.5,24.0,32.0,95.0,3675/1698/1485,OPEL KARL,215.0,1.0,Superutilitario
4506,PEUGEOT,8283.0,5,Gasolina,delantera,68.0,Manual automatizada,5,97.0,833.3,4.2,24.0,35.0,95.0,3475/1615/1460,PEUGEOT 108,196.0,1.0,Superutilitario
4508,PEUGEOT,8529.0,5,Gasolina,delantera,68.0,Manual automatizada,5,97.0,833.3,4.2,24.0,35.0,95.0,3475/1615/1460,PEUGEOT 108,196.0,1.0,Superutilitario
4512,PEUGEOT,9605.0,5,Gasolina,delantera,69.0,Manual automatizada,5,97.0,833.3,4.2,24.0,35.0,95.0,3475/1615/1460,PEUGEOT 108,196.0,1.0,Superutilitario
5097,SKODA,7435.0,3,Gasolina,delantera,60.0,Manual automatizada,5,100.0,795.5,4.4,24.0,35.0,95.0,3563/1645/1478,SKODA CITIGO,251.0,1.0,Superutilitario


In [881]:
prueba['cambio'] = prueba['cambio'].apply(str.strip)

In [883]:
prueba[prueba['cambio']=='Manual automatizada']

,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,capacidad deposito,maletero,tipo_marca,Segmento,longitud
1554,9700.0,5,Gasolina,delantera,68.0,Manual automatizada,5,97.0,833.3,4.2,24.0,35.0,196.0,1.0,Superutilitario,3466.0
1852,11301.0,5,Diesel,delantera,90.0,Manual automatizada,6,92.0,1.388.9,3.6,36.0,50.0,510.0,1.0,Compacto,4501.0
1862,9189.0,5,Gasolina,delantera,90.0,Manual automatizada,5,110.0,1.020.4,4.9,36.0,50.0,320.0,1.0,Utilitario/Polivalente,4069.0
1867,10546.0,5,Diesel,delantera,90.0,Manual automatizada,5,92.0,1.388.9,3.6,36.0,50.0,320.0,1.0,Utilitario/Polivalente,4069.0
1871,13131.0,3,Gasolina,delantera,82.0,Manual automatizada,5,99.0,1.162.8,4.3,24.0,50.0,285.0,1.0,Utilitario/Polivalente,3948.0
4467,8420.0,5,Gasolina,delantera,75.0,Manual automatizada,5,103.0,711.1,4.5,24.0,32.0,215.0,1.0,Superutilitario,3675.0
4506,8283.0,5,Gasolina,delantera,68.0,Manual automatizada,5,97.0,833.3,4.2,24.0,35.0,196.0,1.0,Superutilitario,3475.0
4508,8529.0,5,Gasolina,delantera,68.0,Manual automatizada,5,97.0,833.3,4.2,24.0,35.0,196.0,1.0,Superutilitario,3475.0
4512,9605.0,5,Gasolina,delantera,69.0,Manual automatizada,5,97.0,833.3,4.2,24.0,35.0,196.0,1.0,Superutilitario,3475.0
5097,7435.0,3,Gasolina,delantera,60.0,Manual automatizada,5,100.0,795.5,4.4,24.0,35.0,251.0,1.0,Superutilitario,3563.0


In [886]:
cambios_automaticos = ['Automática secuencial', 'Directo. sin caja de cambios', 'Automática continua. secuencial', 'Automática', 'Automática continua']

In [887]:
def orden_cambios(row):
    if row['cambio'] in cambios_automaticos:
        return 'automatico'
    elif (row['cambio']=='Manual automatizada'):
        return row['cambio']
    else:
        return 'manual'

In [888]:
prueba['cambio_marchas'] = prueba.apply(orden_cambios,axis=1)

In [891]:
prueba.cambio = prueba.cambio_marchas

In [892]:
prueba.drop('cambio_marchas', axis=1,inplace=True)

In [893]:
prueba.head()

,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,capacidad deposito,maletero,tipo_marca,Segmento,longitud
0,35093.0,2,Gasolina,trasera,170.0,manual,6,148.0,703.1,6.4,24.0,45.0,140.0,1.0,Deportivo bajo,4054.0
1,35093.0,2,Gasolina,trasera,170.0,manual,6,148.0,703.1,6.4,24.0,45.0,140.0,1.0,Deportivo bajo,4054.0
2,35093.0,2,Gasolina,trasera,170.0,manual,6,148.0,703.1,6.4,24.0,45.0,140.0,1.0,Deportivo bajo,4054.0
3,35093.0,2,Gasolina,trasera,170.0,manual,6,148.0,703.1,6.4,24.0,45.0,140.0,1.0,Deportivo bajo,4054.0
4,36511.0,2,Gasolina,trasera,170.0,manual,6,148.0,703.1,6.4,24.0,45.0,140.0,1.0,Deportivo bajo,4054.0


### Codificamos el tipo de tracción Delantera->1; Trasera 0.5; Total->0


array(['trasera', 'delantera', 'total'], dtype=object)

In [902]:
def cod_traccion(row):
    if(row['traccion']=='delantera'):
        return 1
    elif(row['traccion']=='trasera'):
        return 0.5
    else:
        return 0

In [903]:
prueba['cod_tra'] = prueba.apply(cod_traccion,axis=1)

In [905]:
prueba['traccion'] = prueba['cod_tra']

In [918]:
prueba.drop('cod_tra',axis=1,inplace=True)

### Codificamos combustible:

#### Vamos a codificar los tipos de combustible (realmente, energía) con las que trabaja el veícuhlo. Tratamos de ordenar por limpieza medioambiental, comenzando por eléctricos puros hasta llegar a los vehículos diésel.

In [908]:
prueba.combustible.unique()

[Gasolina, Gasolina/gas, Diesel, Diesel y corriente eléctrica, Corriente eléctrica, Gasolina y corriente eléctrica]
Categories (6, object): [Gasolina, Gasolina/gas, Diesel, Diesel y corriente eléctrica, Corriente eléctrica, Gasolina y corriente eléctrica]

In [909]:
hibridos = ['Diesel y corriente eléctrica', 'Gasolina y corriente eléctrica']

In [910]:
prueba.combustible = prueba.combustible.apply(str.strip)

In [911]:
def cod_traccion(row):
    if(row['combustible']=='Corriente eléctrica'):
        return 1
    elif(row['combustible'] in hibridos):
        return 0.75
    elif(row['combustible']=='Gasolina/gas'):
        return 0.5
    elif(row['combustible']=='Gasolina'):
        return 0.25
    else:
        return 0

In [915]:
prueba['cod_comb'] = prueba.apply(cod_traccion,axis=1)

In [917]:
prueba['combustible'] = prueba['cod_comb']
prueba.drop('cod_comb',axis=1,inplace=True)
prueba.head()

,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,garantia,capacidad deposito,maletero,tipo_marca,Segmento,longitud,cod_tra
0,35093.0,2,0.25,0.5,170.0,manual,6,148.0,703.1,6.4,24.0,45.0,140.0,1.0,Deportivo bajo,4054.0,0.25
1,35093.0,2,0.25,0.5,170.0,manual,6,148.0,703.1,6.4,24.0,45.0,140.0,1.0,Deportivo bajo,4054.0,0.25
2,35093.0,2,0.25,0.5,170.0,manual,6,148.0,703.1,6.4,24.0,45.0,140.0,1.0,Deportivo bajo,4054.0,0.25
3,35093.0,2,0.25,0.5,170.0,manual,6,148.0,703.1,6.4,24.0,45.0,140.0,1.0,Deportivo bajo,4054.0,0.25
4,36511.0,2,0.25,0.5,170.0,manual,6,148.0,703.1,6.4,24.0,45.0,140.0,1.0,Deportivo bajo,4054.0,0.25


### Codificamos el cambio de marchas 